# **Online Query: Retrieval & Reasoning**

This notebook implements the **Online Query stage**, focused on *retrieval and reasoning*.  

At this stage, the system connects to a **persisted ChromaDB vector store** generated during the ingestion pipeline, retrieves the most relevant context for a user query, and leverages a Large Language Model (LLM) to generate **grounded, context-aware answers**.

## **Objectives of This Notebook**

The main goals of this stage are:

- Load and query an existing vector database (`db/chroma`);
- Validate the behavior of the retriever over real user questions;
- Evaluate prompt construction and context injection strategies;
- Compare **RAG-based answers** against **LLM-only responses**;
- Validate the full Retrieval-Augmented Generation (RAG) pipeline in interactive mode.

## **How to Use This Notebook**

This notebook is designed for **interactive experimentation**.

Each question is executed individually and displayed alongside:
- the answer produced by the RAG pipeline,
- the answer produced by the LLM without retrieval,
- and (optionally) the retrieved context used during generation.

This structure allows direct qualitative comparison and facilitates debugging and iterative refinement.

## **Notebook Setup and Autoreload**

In [95]:
# --- Notebook setup and autoreload configuration ---
# This cell runs the initial setup script and enables the autoreload extension.
# The autoreload feature ensures that updates made to imported modules (e.g., in src/)
# are automatically reloaded without restarting the kernel.
%run notebook_setup.py

%load_ext autoreload

%autoreload 2

2026-01-17T10:59:35 | INFO | NOTEBOOK | Notebook environment configured successfully
2026-01-17T10:59:35 | INFO | NOTEBOOK | Project root: /home/ilfn/datascience/workspace/rag-movie-plots
2026-01-17T10:59:35 | DEBUG | NOTEBOOK | Added to sys.path:
  - /home/ilfn/datascience/workspace/rag-movie-plots/src
  - /home/ilfn/datascience/workspace/rag-movie-plots/src/backend
2026-01-17T10:59:35 | DEBUG | NOTEBOOK | PYTHONPATH: /home/ilfn/datascience/workspace/rag-movie-plots/src
2026-01-17T10:59:35 | DEBUG | NOTEBOOK | Current working directory: /home/ilfn/datascience/workspace/rag-movie-plots/notebooks


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## **Building the Online RAG Components**

In this section, we instantiate the core components required for online querying:
- **Retriever**: interfaces with the persisted `ChromaDB vector store`;
- **PromptBuilder**: assembles prompts by combining retrieved context and user questions;
- **LLMClient**: provides access to the underlying language model;
- **ChatRAG**: orchestrates retrieval, prompt construction, and generation.

In [96]:
from backend.runtime.retrieval.retriever import Retriever
from backend.runtime.chat.chat_rag import ChatRAG
from backend.infra.llm_client import LLMClient
from backend.runtime.chat.prompt_builder import PromptBuilder

# 1) Load retriever (Chroma persisted)
retriever = Retriever()

# 2) Build ChatRAG using the modular components
llm_client = LLMClient()
prompt_builder = PromptBuilder()

chat = ChatRAG(
    retriever=retriever,
    llm_client=llm_client,
    prompt_builder=prompt_builder
)

2026-01-17T10:59:35 | INFO | RETRIEVER | Loading vector store: /home/ilfn/datascience/workspace/rag-movie-plots/db/chroma
2026-01-17T10:59:35 | INFO | RETRIEVER | Embedding model: text-embedding-3-small
2026-01-17T10:59:35 | INFO | RETRIEVER | top_k=10 | use_threshold=False | distance_threshold=0.35
2026-01-17T10:59:35 | INFO | RETRIEVER | Vector store metadata: {'hnsw:space': 'cosine'}


At this point, the full online RAG pipeline is ready.

The system can now:
1. Receive a user question;
2. Retrieve semantically relevant chunks;
3. Build a grounded prompt;
4. Generate an answer conditioned on retrieved evidence.

## **Answer Comparison Utility**

To facilitate qualitative evaluation, we define a helper function that displays:

- the user question,
- the RAG-based answer,
- the LLM-only answer (no context, no RAG prompt),
- and optionally the retrieved context.

This visualization makes hallucinations, omissions, and grounding differences immediately visible.

In [97]:
from IPython.display import display, Markdown

def display_compare(
    question: str,
    rag: str,
    llm_only: str,
    context: str | None = None
):
    display(Markdown(f"## Question\n\n{question}"))


    display(Markdown(
f"""<details>
<summary><b>RAG Answer</b></summary>

{rag if rag else "_No RAG answer available._"}

</details>"""
    ))

    display(Markdown(
f"""<details>
<summary><b>LLM-only Answer (no context, no RAG prompt)</b></summary>

{llm_only if llm_only else "_No LLM-only answer available._"}

</details>"""
    ))

    if context is not None:
        display(Markdown(
f"""<details>
<summary><b>Retrieved Context (debug)</b></summary>

{context}

</details>"""
        ))

    display(Markdown("---"))

The function below executes a single question through two parallel paths:

1. **RAG path**: retrieval -> prompt construction -> generation;
2. **LLM-only path**: direct generation without any external context.

The outputs are displayed side-by-side for comparison.

In [98]:
def ask_compare(question: str, show_context: bool = False):
    # RAG
    rag_out = chat.run(question)
    rag_answer = rag_out.get("answer", "")
    rag_context = rag_out.get("context", "")

    # LLM-only
    llm_only_answer = llm_client.generate(question)

    display_compare(
        question=question,
        rag=rag_answer,
        llm_only=llm_only_answer,
        context=rag_context if show_context else None
    )

### **Query 1 - Ambiguous Entity Resolution**

This query intentionally uses an ambiguous movie title to evaluate how the system handles **entity ambiguity** when multiple films share the same name.

The goal is to observe whether retrieval provides sufficient context to disambiguate the correct movie and prevent narrative mixing or hallucinated facts.

In [99]:
ask_compare(
    "Who directed Titanic and what is the movie about?", 
    show_context=True
)

2026-01-17T10:59:36 | INFO | CHAT_RAG | ChatRAG started | question='Who directed Titanic and what is the movie about?'
2026-01-17T10:59:36 | DEBUG | openai._base_client | Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'idempotency_key': 'stainless-python-retry-c0f48e29-9254-4c51-bd9c-a5a87d2bb543', 'post_parser': <function Embeddings.create.<locals>.parser at 0x7cd80faf9940>, 'json_data': {'input': [[15546, 15910, 89862, 323, 1148, 374, 279, 5818, 922, 30]], 'model': 'text-embedding-3-small', 'encoding_format': 'base64'}}
2026-01-17T10:59:36 | DEBUG | openai._base_client | Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2026-01-17T10:59:36 | DEBUG | httpcore.connection | connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
2026-01-17T10:59:36 | DEBUG | httpcore.connection | connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7cd84d84e420>
2026-01-17T10:59:36 | DEBUG

## Question

Who directed Titanic and what is the movie about?

<details>
<summary><b>RAG Answer</b></summary>

## Summary
The movie **Titanic** (1997), directed by **James Cameron**, is a historical epic and disaster film that follows the ill-fated maiden voyage of the RMS Titanic. The plot centers on the romance between **Rose DeWitt Bukater**, a young first-class passenger, and **Jack Dawson**, a penniless artist, amidst the backdrop of the ship's tragic sinking.

## Analysis
**Titanic** explores themes of love, class disparity, and tragedy. Rose is pressured into an engagement that she does not desire, leading her to contemplate suicide before meeting Jack, who helps her find a sense of freedom and love. Their relationship develops against the societal expectations of their respective classes, culminating in the chaos of the ship's sinking.

## Evidence
- The film is set in **1912** and begins in **Southampton**, where **Rose DeWitt Bukater**, her fiancé **Cal Hockley**, and her mother board the Titanic.
- Rose struggles with her engagement, feeling trapped by her family's financial issues, and contemplates suicide before being saved by Jack Dawson.
- The narrative intertwines their romance with the impending disaster of the Titanic, which ultimately leads to the ship's tragic sinking.

Source Acknowledgment:
- Titanic (1997)

</details>

<details>
<summary><b>LLM-only Answer (no context, no RAG prompt)</b></summary>

"Titanic" was directed by James Cameron and was released in 1997. The film is a romantic drama that tells the story of the ill-fated maiden voyage of the RMS Titanic, which sank after hitting an iceberg in April 1912. 

The narrative centers around two fictional characters, Jack Dawson (played by Leonardo DiCaprio) and Rose DeWitt Bukater (played by Kate Winslet), who come from different social classes. Jack is a penniless artist, while Rose is a young woman from a wealthy family who feels trapped in her privileged life and is engaged to a wealthy but controlling fiancé. Their love story unfolds against the backdrop of the ship's tragic sinking, exploring themes of love, sacrifice, and the class divide. The film combines historical events with a fictional romance, and it became one of the highest-grossing films of all time, winning numerous awards, including 11 Academy Awards.

</details>

<details>
<summary><b>Retrieved Context (debug)</b></summary>

chunk_id: 34183_0
doc_id: 34183
Title: Blood and Steel
Release Year: 2011
Genre: Not specified
Director: Not specified
Cast: Not specified
Origin/Ethnicity: Russian
Content (Plot):
<<<
The series follows the lives of the people who made the Titanic, from the workers who built it to its rich financiers. Dr. Mark Muir, an engineer and metallurgist, convinces American tycoon J.P. Morgan to hire him for the biggest shipping project in the world, the construction of the RMS Titanic at Belfast's Harland and Wolff shipyard. Mark is, in truth, a Belfast native born Marcus Malone. Now, with a new name and identity, he tries to hide his heritage from his employers, as he is Catholic and his employers, the Protestant elite that rule Belfast, dislike Catholics.
While working there, Mark falls in love with Sofia Silvestri, an Italian immigrant. However, during the construction of the Titanic, tensions rise between the lower-class workers and the rich elite. More setbacks stall the construction: Harland and Wolff want to save costs and use cheaper materials, the workers wish to form a trade union, the women suffrage movement heats up in the UK, and the pro Home Rule and pro-Unionist groups battle each other. Mark attempts to deal with these while trying to escape his past.
>>>

chunk_id: 12857_0
doc_id: 12857
Title: Titanic
Release Year: 1996
Genre: Biography
Director: Robert Lieberman
Cast: George C. Scott, Eva Marie Saint, Peter Gallagher, Catherine Zeta-Jones
Origin/Ethnicity: American
Content (Plot):
<<<
Titanic follows three main story threads.
Isabella Paradine is traveling on the Titanic to join her husband after attending her aunt's funeral in England. On the Titanic, she meets Wynn Park, her former lover. She falls in love with him again, and after a brief affair, she sends her husband a wireless saying they cannot be together anymore (despite their daughter). When the ship starts sinking, Isabella reluctantly leaves Wynn when he forces her to board a lifeboat. As the boat is lowered, Isabella confesses a long kept secret that her daughter Claire is actually Wynn's. Later on board the RMS Carpathia she is grief-stricken when she finds Wynn's lifeless body on deck, having died of hypothermia, but luckily, when the Carpathia reaches New York she is reunited with her family who are blissfully unaware of Isabella's tryst because the telegram was never sent out due to the sinking.
>>>

chunk_id: 6008_0
doc_id: 6008
Title: Titanic
Release Year: 1953
Genre: Biography
Director: Jean Negulesco
Cast: Barbara Stanwyck, Clifton Webb, Robert Wagner, Thelma Ritter, Richard Basehart
Origin/Ethnicity: American
Content (Plot):
<<<
At the last minute, a wealthy American expatriate in Europe, Richard Sturges (Clifton Webb), buys a steerage-class ticket (the lowest class) for the maiden voyage of the RMS Titanic from a Basque immigrant. Once aboard he seeks out his runaway wife, Julia (Barbara Stanwyck). He discovers she is trying to take their two unsuspecting children, 18-year-old Annette (Audrey Dalton) and ten-year-old Norman (Harper Carter), to her hometown of Mackinac, Michigan, to raise as down-to-earth Americans rather than rootless elitists like Richard himself.
As the ship prepares for departure, her captain, E. J. Smith (Brian Aherne), receives a hint from the shipping company representative that a record-setting speedy passage would be welcomed.
Other passengers include a wealthy woman of a working-class origin (based on a real-life Titanic survivor Molly Brown), Maude Young (Thelma Ritter); social-climbing Earl Meeker (Allyn Joslyn); a 20-year-old Purdue University tennis player, Gifford "Giff" Rogers (Robert Wagner); and George S. Healey (Richard Basehart), a Catholic priest who has been defrocked for alcoholism.
>>>

chunk_id: 18117_0
doc_id: 18117
Title: Atlantic
Release Year: 1929
Genre: Drama
Director: Ewald André Dupont
Cast: Franklin Dyall, Madeleine Carroll
Origin/Ethnicity: British
Content (Plot):
<<<
Atlantic is a drama film based on the RMS Titanic and set aboard a fictional ship, called the Atlantic. The main plotline revolves around a man who has a shipboard affair with a fellow passenger, which is eventually discovered by his wife. The ship also has aboard an elderly couple, the Rools, who are on their anniversary cruise. Midway across the Atlantic Ocean, the Atlantic strikes an iceberg and is damaged to the point where it is sinking into the Atlantic. A shortage of lifeboats causes the crew to only allow women and children in (though the captain allows a few men to take to the last remaining boats as the disaster reaches its zenith) and many couples are separated. Mrs. Rool refuses to leave her husband and after the boats are gone all the passengers gather on the deck and sing "Nearer, My God, to Thee" as the Atlantic sinks into the ocean. The final scenes depict a group of passengers saying the Lord's Prayer in a flooding lounge.
>>>

chunk_id: 19401_0
doc_id: 19401
Title: A Night to Remember
Release Year: 1958
Genre: Drama
Director: Roy Ward Baker
Cast: Kenneth More, Ronald Allen, Honor Blackman
Origin/Ethnicity: British
Content (Plot):
<<<
In 1912, the Titanic is the largest vessel afloat and is widely believed to be unsinkable. Passengers aboard for her maiden voyage are the cream of American and British society. Boarding are first class passengers Sir Richard and Lady Richard, second class passengers Mr. Clarke and Mrs. Clarke, a young newly wed couple, and steerage passengers Mr. Murphy, Mr. Gallagher and Mr. James Farrel. Second Officer Charles Lightoller is also readying for the voyage. On 10 April, Titanic sails out to sea.
On 14 April, at sea, the ship receives a number of ice warnings from other steamers. Only a few of the messages are relayed to Captain Edward J. Smith, who orders a lookout, but does not slow the ship or consider changing course.
>>>

chunk_id: 13153_1
doc_id: 13153
Title: Titanic
Release Year: 1997
Genre: Historical Epic, Disaster
Director: James Cameron
Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Frances Fisher, Victor Garber, Kathy Bates, Bill Paxton, Gloria Stuart, David Warner, Suzy Amis
Origin/Ethnicity: American
Content (Plot):
<<<
In 1912 Southampton, 17-year-old first-class passenger Rose DeWitt Bukater, her fiancé Cal Hockley, and her mother Ruth board the luxurious Titanic. Ruth emphasizes that Rose's marriage will resolve their family's financial problems and retain their high-class persona. Distraught over the engagement, Rose considers suicide by jumping from the stern; Jack Dawson, a penniless artist, intervenes and discourages her. Discovered with Jack, Rose tells a concerned Cal that she was peering over the edge and Jack saved her from falling. When Cal becomes indifferent, she suggests to him that Jack deserves a reward. He invites Jack to dine with them in first class the following night. Jack and Rose develop a tentative friendship, despite Cal and Ruth being wary of him. Following dinner, Rose secretly joins Jack at a party in third class.
>>>

chunk_id: 16392_1
doc_id: 16392
Title: Titanic 3D
Release Year: 2012
Genre: Drama
Director: James Cameron
Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates, Francis Fisher, Gloria Stuart, Bill Paxton, Bernard Hill, David Warner, Victor Garber, Jonathan Hyde, Eric Braeden, Bernard Fox, Suzy Amis, Danny Nucci, Lewis Abernathy, Nicholas Cascone
Origin/Ethnicity: American
Content (Plot):
<<<
In 1912 Southampton, 17-year-old first-class passenger Rose DeWitt Bukater, her fiancé Cal Hockley, and her mother Ruth board the luxurious Titanic. Ruth emphasizes that Rose's marriage will resolve their family's financial problems and retain their high-class persona. Distraught over the engagement, Rose considers suicide by jumping from the stern; Jack Dawson, a penniless artist, intervenes and discourages her. Discovered with Jack, Rose tells a concerned Cal that she was peering over the edge and Jack saved her from falling. When Cal becomes indifferent, she suggests to him that Jack deserves a reward. He invites Jack to dine with them in first class the following night. Jack and Rose develop a tentative friendship, despite Cal and Ruth being wary of him. Following dinner, Rose secretly joins Jack at a party in third class.
>>>

chunk_id: 9758_1
doc_id: 9758
Title: Raise the Titanic
Release Year: 1980
Genre: Action
Director: Jerry Jameson
Cast: Jason Robards, Alec Guinness, Anne Archer
Origin/Ethnicity: American
Content (Plot):
<<<
The CIA and Pitt soon find out that boxes of the raw mineral were loaded onto the Belfast-built RMS Titanic by an American in April 1912. A search is then conducted in the North Atlantic to locate the sunken ocean liner. The search team is aided by one of the Titanic's last survivors (Alec Guinness) who explains he was also the last person to see the American alive. Just before the Titanic foundered, the sailor said he locked the man inside the ship's vault containing the boxes of mineral, his last words being "thank God for Southby!" At this point it is decided that the only way to get a hold of the byzanium is to literally "raise the Titanic" from the ocean floor. Pitt comes up with a salvage plan that Sandecker presents to the president. The President signs off on the plan and Pitt is put in charge of the operation.
>>>

chunk_id: 9758_3
doc_id: 9758
Title: Raise the Titanic
Release Year: 1980
Genre: Action
Director: Jerry Jameson
Cast: Jason Robards, Alec Guinness, Anne Archer
Origin/Ethnicity: American
Content (Plot):
<<<
Eventually, the rusting Titanic is brought to the surface by using explosives to break the hull loose from the bottom suction and compressed air tanks to fill buoyancy aids. During the ascent, the Deep Quest safely breaks away from the ship.
When Prevlov, who has been aboard a nearby Soviet spy ship, sees the Titanic, he arranges a fake distress call to draw the American naval escorts away from the operation. He then meets with Sandecker, Pitt and Seagram aboard their vessel. He tells them that his government knows all about the mineral and challenges them for both salvage of the Titanic as well as ownership of the ore, claiming it was illegally taken from Russian soil. Prevlov says that if there is to be a "superior weapon" made from the mineral, then "Russia must have it!" Sandecker tells Prevlov that they knew he was coming and what he would threaten them with. Pitt then escorts him to the deck where U.S. fighter jets and a nuclear attack submarine have arrived to protect the Titanic from their attempted piracy. Prevlov leaves in defeat.
>>>

chunk_id: 9758_2
doc_id: 9758
Title: Raise the Titanic
Release Year: 1980
Genre: Action
Director: Jerry Jameson
Cast: Jason Robards, Alec Guinness, Anne Archer
Origin/Ethnicity: American
Content (Plot):
<<<
At this time the Soviet KGB station chief in Washington D.C., Andre Prevlov (Bo Brundin), is receiving bits of information on the project and leaks them to a reporter, Dana Archibald (Anne Archer), who is also Seagram's lover as well as a former girlfriend of Pitt's. The story blows the project's secret cover and Sandecker must hold a press conference to explain why the ship is being raised. Questions are raised about byzanium, but are not answered.
After a lengthy search, the Titanic is located and the search team, with help from the U.S. Navy, begins the dangerous job of raising the ship from the seabed. One of the submersibles, Starfish, experiences a cabin flood and implodes. Another submersible, the Deep Quest, is attempting to clear debris from one of the upper decks when it suddenly tears free of its supports, crashes through the skylight above the main staircase and becomes jammed. Pitt decides they must attempt to raise the ship before the Deep Quest crew suffocates.
>>>

</details>

---

### **Query 2 - Explicit Disambiguation Prompt**

This variation explicitly acknowledges that multiple movies share the same title.

By doing so, the query tests whether retrieval improves factual grounding when ambiguity is surfaced in the prompt itself, and whether the system selects a coherent, evidence-backed interpretation.

In [100]:
ask_compare(
    "Who directed a movie with the Title Titanic and what is the movie about?", 
    show_context=True
)

2026-01-17T10:59:47 | INFO | CHAT_RAG | ChatRAG started | question='Who directed a movie with the Title Titanic and what is the movie about?'
2026-01-17T10:59:47 | DEBUG | openai._base_client | Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'idempotency_key': 'stainless-python-retry-6cb1e798-42b5-4e8c-a9d7-6f978cf8ef37', 'post_parser': <function Embeddings.create.<locals>.parser at 0x7cd80fafa3e0>, 'json_data': {'input': [[15546, 15910, 264, 5818, 449, 279, 11106, 89862, 323, 1148, 374, 279, 5818, 922, 30]], 'model': 'text-embedding-3-small', 'encoding_format': 'base64'}}
2026-01-17T10:59:47 | DEBUG | openai._base_client | Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2026-01-17T10:59:47 | DEBUG | httpcore.connection | close.started
2026-01-17T10:59:47 | DEBUG | httpcore.connection | close.complete
2026-01-17T10:59:47 | DEBUG | httpcore.connection | connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_opti

## Question

Who directed a movie with the Title Titanic and what is the movie about?

<details>
<summary><b>RAG Answer</b></summary>

## Summary
The title **Titanic** is associated with multiple films directed by different individuals. The most notable ones include:

1. **Titanic (1996)** directed by **Robert Lieberman**: This film follows three main storylines, focusing on Isabella Paradine, who has an affair with her former lover Wynn Park while on the Titanic. As the ship sinks, she faces the consequences of her choices and ultimately reunites with her family, unaware of the tragedy that has befallen Wynn.

2. **Titanic (1953)** directed by **Jean Negulesco**: This version centers on Richard Sturges, a wealthy American expatriate, who boards the Titanic to find his runaway wife, Julia. The film explores themes of family and class as the ship embarks on its ill-fated maiden voyage.

3. **Titanic (1997)** directed by **James Cameron**: This epic film tells the love story of Rose DeWitt Bukater and Jack Dawson aboard the Titanic. It highlights class struggles and personal sacrifice against the backdrop of the ship's tragic sinking.

## Analysis
Each film titled **Titanic** presents a unique narrative, but they all share the common setting of the RMS Titanic and explore themes of love, loss, and societal class distinctions. The 1997 version is particularly renowned for its historical accuracy and emotional depth, while the 1953 and 1996 versions offer different perspectives on personal relationships during the disaster.

## Evidence
- **Titanic (1996)**: "Titanic follows three main story threads... Isabella Paradine is traveling on the Titanic... she meets Wynn Park, her former lover... As the ship starts sinking, Isabella reluctantly leaves Wynn..."
- **Titanic (1953)**: "At the last minute, a wealthy American expatriate in Europe, Richard Sturges... seeks out his runaway wife, Julia... discovers she is trying to take their two unsuspecting children..."
- **Titanic (1997)**: "In 1912 Southampton, 17-year-old first-class passenger Rose DeWitt Bukater... Jack Dawson, a penniless artist, intervenes and discourages her... Jack and Rose develop a tentative friendship..."

## Source Acknowledgment
- Titanic (1996)
- Titanic (1953)
- Titanic (1997)

</details>

<details>
<summary><b>LLM-only Answer (no context, no RAG prompt)</b></summary>

The movie "Titanic," released in 1997, was directed by James Cameron. It is a romantic drama that tells the story of a fictional love affair between two characters, Jack Dawson (played by Leonardo DiCaprio) and Rose DeWitt Bukater (played by Kate Winslet), set against the backdrop of the real-life sinking of the RMS Titanic in 1912. 

The film explores themes of love, class disparity, and tragedy, as Jack, a poor artist, and Rose, a wealthy young woman, meet aboard the ill-fated ship. Their romance unfolds amidst the opulence of the Titanic and the societal constraints of their time. The film also depicts the ship's tragic voyage and the chaos that ensues as it strikes an iceberg and sinks. "Titanic" was a massive critical and commercial success, winning numerous awards, including 11 Academy Awards, and is known for its groundbreaking visual effects and emotional storytelling.

</details>

<details>
<summary><b>Retrieved Context (debug)</b></summary>

chunk_id: 18117_0
doc_id: 18117
Title: Atlantic
Release Year: 1929
Genre: Drama
Director: Ewald André Dupont
Cast: Franklin Dyall, Madeleine Carroll
Origin/Ethnicity: British
Content (Plot):
<<<
Atlantic is a drama film based on the RMS Titanic and set aboard a fictional ship, called the Atlantic. The main plotline revolves around a man who has a shipboard affair with a fellow passenger, which is eventually discovered by his wife. The ship also has aboard an elderly couple, the Rools, who are on their anniversary cruise. Midway across the Atlantic Ocean, the Atlantic strikes an iceberg and is damaged to the point where it is sinking into the Atlantic. A shortage of lifeboats causes the crew to only allow women and children in (though the captain allows a few men to take to the last remaining boats as the disaster reaches its zenith) and many couples are separated. Mrs. Rool refuses to leave her husband and after the boats are gone all the passengers gather on the deck and sing "Nearer, My God, to Thee" as the Atlantic sinks into the ocean. The final scenes depict a group of passengers saying the Lord's Prayer in a flooding lounge.
>>>

chunk_id: 34183_0
doc_id: 34183
Title: Blood and Steel
Release Year: 2011
Genre: Not specified
Director: Not specified
Cast: Not specified
Origin/Ethnicity: Russian
Content (Plot):
<<<
The series follows the lives of the people who made the Titanic, from the workers who built it to its rich financiers. Dr. Mark Muir, an engineer and metallurgist, convinces American tycoon J.P. Morgan to hire him for the biggest shipping project in the world, the construction of the RMS Titanic at Belfast's Harland and Wolff shipyard. Mark is, in truth, a Belfast native born Marcus Malone. Now, with a new name and identity, he tries to hide his heritage from his employers, as he is Catholic and his employers, the Protestant elite that rule Belfast, dislike Catholics.
While working there, Mark falls in love with Sofia Silvestri, an Italian immigrant. However, during the construction of the Titanic, tensions rise between the lower-class workers and the rich elite. More setbacks stall the construction: Harland and Wolff want to save costs and use cheaper materials, the workers wish to form a trade union, the women suffrage movement heats up in the UK, and the pro Home Rule and pro-Unionist groups battle each other. Mark attempts to deal with these while trying to escape his past.
>>>

chunk_id: 12857_0
doc_id: 12857
Title: Titanic
Release Year: 1996
Genre: Biography
Director: Robert Lieberman
Cast: George C. Scott, Eva Marie Saint, Peter Gallagher, Catherine Zeta-Jones
Origin/Ethnicity: American
Content (Plot):
<<<
Titanic follows three main story threads.
Isabella Paradine is traveling on the Titanic to join her husband after attending her aunt's funeral in England. On the Titanic, she meets Wynn Park, her former lover. She falls in love with him again, and after a brief affair, she sends her husband a wireless saying they cannot be together anymore (despite their daughter). When the ship starts sinking, Isabella reluctantly leaves Wynn when he forces her to board a lifeboat. As the boat is lowered, Isabella confesses a long kept secret that her daughter Claire is actually Wynn's. Later on board the RMS Carpathia she is grief-stricken when she finds Wynn's lifeless body on deck, having died of hypothermia, but luckily, when the Carpathia reaches New York she is reunited with her family who are blissfully unaware of Isabella's tryst because the telegram was never sent out due to the sinking.
>>>

chunk_id: 6008_0
doc_id: 6008
Title: Titanic
Release Year: 1953
Genre: Biography
Director: Jean Negulesco
Cast: Barbara Stanwyck, Clifton Webb, Robert Wagner, Thelma Ritter, Richard Basehart
Origin/Ethnicity: American
Content (Plot):
<<<
At the last minute, a wealthy American expatriate in Europe, Richard Sturges (Clifton Webb), buys a steerage-class ticket (the lowest class) for the maiden voyage of the RMS Titanic from a Basque immigrant. Once aboard he seeks out his runaway wife, Julia (Barbara Stanwyck). He discovers she is trying to take their two unsuspecting children, 18-year-old Annette (Audrey Dalton) and ten-year-old Norman (Harper Carter), to her hometown of Mackinac, Michigan, to raise as down-to-earth Americans rather than rootless elitists like Richard himself.
As the ship prepares for departure, her captain, E. J. Smith (Brian Aherne), receives a hint from the shipping company representative that a record-setting speedy passage would be welcomed.
Other passengers include a wealthy woman of a working-class origin (based on a real-life Titanic survivor Molly Brown), Maude Young (Thelma Ritter); social-climbing Earl Meeker (Allyn Joslyn); a 20-year-old Purdue University tennis player, Gifford "Giff" Rogers (Robert Wagner); and George S. Healey (Richard Basehart), a Catholic priest who has been defrocked for alcoholism.
>>>

chunk_id: 19401_0
doc_id: 19401
Title: A Night to Remember
Release Year: 1958
Genre: Drama
Director: Roy Ward Baker
Cast: Kenneth More, Ronald Allen, Honor Blackman
Origin/Ethnicity: British
Content (Plot):
<<<
In 1912, the Titanic is the largest vessel afloat and is widely believed to be unsinkable. Passengers aboard for her maiden voyage are the cream of American and British society. Boarding are first class passengers Sir Richard and Lady Richard, second class passengers Mr. Clarke and Mrs. Clarke, a young newly wed couple, and steerage passengers Mr. Murphy, Mr. Gallagher and Mr. James Farrel. Second Officer Charles Lightoller is also readying for the voyage. On 10 April, Titanic sails out to sea.
On 14 April, at sea, the ship receives a number of ice warnings from other steamers. Only a few of the messages are relayed to Captain Edward J. Smith, who orders a lookout, but does not slow the ship or consider changing course.
>>>

chunk_id: 13153_1
doc_id: 13153
Title: Titanic
Release Year: 1997
Genre: Historical Epic, Disaster
Director: James Cameron
Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Frances Fisher, Victor Garber, Kathy Bates, Bill Paxton, Gloria Stuart, David Warner, Suzy Amis
Origin/Ethnicity: American
Content (Plot):
<<<
In 1912 Southampton, 17-year-old first-class passenger Rose DeWitt Bukater, her fiancé Cal Hockley, and her mother Ruth board the luxurious Titanic. Ruth emphasizes that Rose's marriage will resolve their family's financial problems and retain their high-class persona. Distraught over the engagement, Rose considers suicide by jumping from the stern; Jack Dawson, a penniless artist, intervenes and discourages her. Discovered with Jack, Rose tells a concerned Cal that she was peering over the edge and Jack saved her from falling. When Cal becomes indifferent, she suggests to him that Jack deserves a reward. He invites Jack to dine with them in first class the following night. Jack and Rose develop a tentative friendship, despite Cal and Ruth being wary of him. Following dinner, Rose secretly joins Jack at a party in third class.
>>>

chunk_id: 16392_1
doc_id: 16392
Title: Titanic 3D
Release Year: 2012
Genre: Drama
Director: James Cameron
Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates, Francis Fisher, Gloria Stuart, Bill Paxton, Bernard Hill, David Warner, Victor Garber, Jonathan Hyde, Eric Braeden, Bernard Fox, Suzy Amis, Danny Nucci, Lewis Abernathy, Nicholas Cascone
Origin/Ethnicity: American
Content (Plot):
<<<
In 1912 Southampton, 17-year-old first-class passenger Rose DeWitt Bukater, her fiancé Cal Hockley, and her mother Ruth board the luxurious Titanic. Ruth emphasizes that Rose's marriage will resolve their family's financial problems and retain their high-class persona. Distraught over the engagement, Rose considers suicide by jumping from the stern; Jack Dawson, a penniless artist, intervenes and discourages her. Discovered with Jack, Rose tells a concerned Cal that she was peering over the edge and Jack saved her from falling. When Cal becomes indifferent, she suggests to him that Jack deserves a reward. He invites Jack to dine with them in first class the following night. Jack and Rose develop a tentative friendship, despite Cal and Ruth being wary of him. Following dinner, Rose secretly joins Jack at a party in third class.
>>>

chunk_id: 9758_3
doc_id: 9758
Title: Raise the Titanic
Release Year: 1980
Genre: Action
Director: Jerry Jameson
Cast: Jason Robards, Alec Guinness, Anne Archer
Origin/Ethnicity: American
Content (Plot):
<<<
Eventually, the rusting Titanic is brought to the surface by using explosives to break the hull loose from the bottom suction and compressed air tanks to fill buoyancy aids. During the ascent, the Deep Quest safely breaks away from the ship.
When Prevlov, who has been aboard a nearby Soviet spy ship, sees the Titanic, he arranges a fake distress call to draw the American naval escorts away from the operation. He then meets with Sandecker, Pitt and Seagram aboard their vessel. He tells them that his government knows all about the mineral and challenges them for both salvage of the Titanic as well as ownership of the ore, claiming it was illegally taken from Russian soil. Prevlov says that if there is to be a "superior weapon" made from the mineral, then "Russia must have it!" Sandecker tells Prevlov that they knew he was coming and what he would threaten them with. Pitt then escorts him to the deck where U.S. fighter jets and a nuclear attack submarine have arrived to protect the Titanic from their attempted piracy. Prevlov leaves in defeat.
>>>

chunk_id: 9758_2
doc_id: 9758
Title: Raise the Titanic
Release Year: 1980
Genre: Action
Director: Jerry Jameson
Cast: Jason Robards, Alec Guinness, Anne Archer
Origin/Ethnicity: American
Content (Plot):
<<<
At this time the Soviet KGB station chief in Washington D.C., Andre Prevlov (Bo Brundin), is receiving bits of information on the project and leaks them to a reporter, Dana Archibald (Anne Archer), who is also Seagram's lover as well as a former girlfriend of Pitt's. The story blows the project's secret cover and Sandecker must hold a press conference to explain why the ship is being raised. Questions are raised about byzanium, but are not answered.
After a lengthy search, the Titanic is located and the search team, with help from the U.S. Navy, begins the dangerous job of raising the ship from the seabed. One of the submersibles, Starfish, experiences a cabin flood and implodes. Another submersible, the Deep Quest, is attempting to clear debris from one of the upper decks when it suddenly tears free of its supports, crashes through the skylight above the main staircase and becomes jammed. Pitt decides they must attempt to raise the ship before the Deep Quest crew suffocates.
>>>

chunk_id: 9758_1
doc_id: 9758
Title: Raise the Titanic
Release Year: 1980
Genre: Action
Director: Jerry Jameson
Cast: Jason Robards, Alec Guinness, Anne Archer
Origin/Ethnicity: American
Content (Plot):
<<<
The CIA and Pitt soon find out that boxes of the raw mineral were loaded onto the Belfast-built RMS Titanic by an American in April 1912. A search is then conducted in the North Atlantic to locate the sunken ocean liner. The search team is aided by one of the Titanic's last survivors (Alec Guinness) who explains he was also the last person to see the American alive. Just before the Titanic foundered, the sailor said he locked the man inside the ship's vault containing the boxes of mineral, his last words being "thank God for Southby!" At this point it is decided that the only way to get a hold of the byzanium is to literally "raise the Titanic" from the ocean floor. Pitt comes up with a salvage plan that Sandecker presents to the president. The President signs off on the plan and Pitt is put in charge of the operation.
>>>

</details>

---

### **Query 3 - Narrative and Plot-Based Reasoning**

This query evaluates the system's ability to retrieve and synthesize **narrative-level information**, rather than relying on surface metadata.

The focus is on identifying the central conflict of the story and assessing whether the answer is grounded in retrieved plot descriptions.

In [101]:
ask_compare(
    "What is the main conflict in the movie Titanic?",
    show_context=True
)

2026-01-17T11:00:03 | INFO | CHAT_RAG | ChatRAG started | question='What is the main conflict in the movie Titanic?'
2026-01-17T11:00:03 | DEBUG | openai._base_client | Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'idempotency_key': 'stainless-python-retry-1f81e0af-72da-4c14-9d3a-336d575f1b84', 'post_parser': <function Embeddings.create.<locals>.parser at 0x7cd80fafa5c0>, 'json_data': {'input': [[3923, 374, 279, 1925, 12324, 304, 279, 5818, 89862, 30]], 'model': 'text-embedding-3-small', 'encoding_format': 'base64'}}
2026-01-17T11:00:03 | DEBUG | openai._base_client | Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2026-01-17T11:00:03 | DEBUG | httpcore.connection | close.started
2026-01-17T11:00:03 | DEBUG | httpcore.connection | close.complete
2026-01-17T11:00:03 | DEBUG | httpcore.connection | connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
2026-01-17T11:00:03 | DEBUG | httpcore.connec

## Question

What is the main conflict in the movie Titanic?

<details>
<summary><b>RAG Answer</b></summary>

## Summary
The main conflict in the movie **Titanic** revolves around the personal struggles of the characters amidst the catastrophic sinking of the ship. In the 1997 version directed by James Cameron, the central conflict is primarily between **Rose DeWitt Bukater**, who is trapped in a loveless engagement to **Cal Hockley**, and her burgeoning love for **Jack Dawson**, a penniless artist. This personal conflict is set against the backdrop of the Titanic's tragic fate, which ultimately leads to a struggle for survival as the ship sinks.

In the 1996 version, the conflict centers on **Isabella Paradine**, who rekindles a romance with her former lover **Wynn Park** while on the Titanic, leading to emotional turmoil as she must choose between her family and her love for Wynn. This personal conflict culminates in a tragic revelation and the struggle for survival as the ship sinks.

## Analysis
The conflicts in both versions of **Titanic** highlight themes of love, sacrifice, and societal constraints. The characters face not only the external threat of the ship's sinking but also internal dilemmas regarding their relationships and personal choices. The sinking of the Titanic serves as a catalyst that forces the characters to confront their true feelings and make life-altering decisions.

## Evidence
- In the **1997 version**, Rose's conflict is illustrated by her engagement to Cal and her relationship with Jack, which leads her to consider suicide before Jack intervenes. Their love story unfolds against the backdrop of the ship's tragic sinking.
- In the **1996 version**, Isabella's conflict arises from her affair with Wynn and the emotional turmoil of leaving her husband and daughter behind as the ship sinks. Her ultimate decision to confess a secret about her daughter adds depth to her internal struggle.

Source Acknowledgment:
- Titanic (1996)
- Titanic (1997)

</details>

<details>
<summary><b>LLM-only Answer (no context, no RAG prompt)</b></summary>

The main conflict in the movie "Titanic" revolves around the love story between Jack Dawson and Rose DeWitt Bukater, set against the backdrop of the ill-fated maiden voyage of the RMS Titanic. Their romance faces numerous obstacles, including class differences, societal expectations, and Rose's engagement to the wealthy but controlling Cal Hockley. 

Additionally, the film presents a larger conflict between human ambition and nature, as the Titanic, deemed "unsinkable," tragically collides with an iceberg, leading to a catastrophic disaster. This dual conflict of personal and societal struggles, combined with the impending tragedy of the ship's sinking, drives the narrative and highlights themes of love, sacrifice, and the fragility of life.

</details>

<details>
<summary><b>Retrieved Context (debug)</b></summary>

chunk_id: 12857_0
doc_id: 12857
Title: Titanic
Release Year: 1996
Genre: Biography
Director: Robert Lieberman
Cast: George C. Scott, Eva Marie Saint, Peter Gallagher, Catherine Zeta-Jones
Origin/Ethnicity: American
Content (Plot):
<<<
Titanic follows three main story threads.
Isabella Paradine is traveling on the Titanic to join her husband after attending her aunt's funeral in England. On the Titanic, she meets Wynn Park, her former lover. She falls in love with him again, and after a brief affair, she sends her husband a wireless saying they cannot be together anymore (despite their daughter). When the ship starts sinking, Isabella reluctantly leaves Wynn when he forces her to board a lifeboat. As the boat is lowered, Isabella confesses a long kept secret that her daughter Claire is actually Wynn's. Later on board the RMS Carpathia she is grief-stricken when she finds Wynn's lifeless body on deck, having died of hypothermia, but luckily, when the Carpathia reaches New York she is reunited with her family who are blissfully unaware of Isabella's tryst because the telegram was never sent out due to the sinking.
>>>

chunk_id: 18117_0
doc_id: 18117
Title: Atlantic
Release Year: 1929
Genre: Drama
Director: Ewald André Dupont
Cast: Franklin Dyall, Madeleine Carroll
Origin/Ethnicity: British
Content (Plot):
<<<
Atlantic is a drama film based on the RMS Titanic and set aboard a fictional ship, called the Atlantic. The main plotline revolves around a man who has a shipboard affair with a fellow passenger, which is eventually discovered by his wife. The ship also has aboard an elderly couple, the Rools, who are on their anniversary cruise. Midway across the Atlantic Ocean, the Atlantic strikes an iceberg and is damaged to the point where it is sinking into the Atlantic. A shortage of lifeboats causes the crew to only allow women and children in (though the captain allows a few men to take to the last remaining boats as the disaster reaches its zenith) and many couples are separated. Mrs. Rool refuses to leave her husband and after the boats are gone all the passengers gather on the deck and sing "Nearer, My God, to Thee" as the Atlantic sinks into the ocean. The final scenes depict a group of passengers saying the Lord's Prayer in a flooding lounge.
>>>

chunk_id: 34183_0
doc_id: 34183
Title: Blood and Steel
Release Year: 2011
Genre: Not specified
Director: Not specified
Cast: Not specified
Origin/Ethnicity: Russian
Content (Plot):
<<<
The series follows the lives of the people who made the Titanic, from the workers who built it to its rich financiers. Dr. Mark Muir, an engineer and metallurgist, convinces American tycoon J.P. Morgan to hire him for the biggest shipping project in the world, the construction of the RMS Titanic at Belfast's Harland and Wolff shipyard. Mark is, in truth, a Belfast native born Marcus Malone. Now, with a new name and identity, he tries to hide his heritage from his employers, as he is Catholic and his employers, the Protestant elite that rule Belfast, dislike Catholics.
While working there, Mark falls in love with Sofia Silvestri, an Italian immigrant. However, during the construction of the Titanic, tensions rise between the lower-class workers and the rich elite. More setbacks stall the construction: Harland and Wolff want to save costs and use cheaper materials, the workers wish to form a trade union, the women suffrage movement heats up in the UK, and the pro Home Rule and pro-Unionist groups battle each other. Mark attempts to deal with these while trying to escape his past.
>>>

chunk_id: 9758_3
doc_id: 9758
Title: Raise the Titanic
Release Year: 1980
Genre: Action
Director: Jerry Jameson
Cast: Jason Robards, Alec Guinness, Anne Archer
Origin/Ethnicity: American
Content (Plot):
<<<
Eventually, the rusting Titanic is brought to the surface by using explosives to break the hull loose from the bottom suction and compressed air tanks to fill buoyancy aids. During the ascent, the Deep Quest safely breaks away from the ship.
When Prevlov, who has been aboard a nearby Soviet spy ship, sees the Titanic, he arranges a fake distress call to draw the American naval escorts away from the operation. He then meets with Sandecker, Pitt and Seagram aboard their vessel. He tells them that his government knows all about the mineral and challenges them for both salvage of the Titanic as well as ownership of the ore, claiming it was illegally taken from Russian soil. Prevlov says that if there is to be a "superior weapon" made from the mineral, then "Russia must have it!" Sandecker tells Prevlov that they knew he was coming and what he would threaten them with. Pitt then escorts him to the deck where U.S. fighter jets and a nuclear attack submarine have arrived to protect the Titanic from their attempted piracy. Prevlov leaves in defeat.
>>>

chunk_id: 13153_1
doc_id: 13153
Title: Titanic
Release Year: 1997
Genre: Historical Epic, Disaster
Director: James Cameron
Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Frances Fisher, Victor Garber, Kathy Bates, Bill Paxton, Gloria Stuart, David Warner, Suzy Amis
Origin/Ethnicity: American
Content (Plot):
<<<
In 1912 Southampton, 17-year-old first-class passenger Rose DeWitt Bukater, her fiancé Cal Hockley, and her mother Ruth board the luxurious Titanic. Ruth emphasizes that Rose's marriage will resolve their family's financial problems and retain their high-class persona. Distraught over the engagement, Rose considers suicide by jumping from the stern; Jack Dawson, a penniless artist, intervenes and discourages her. Discovered with Jack, Rose tells a concerned Cal that she was peering over the edge and Jack saved her from falling. When Cal becomes indifferent, she suggests to him that Jack deserves a reward. He invites Jack to dine with them in first class the following night. Jack and Rose develop a tentative friendship, despite Cal and Ruth being wary of him. Following dinner, Rose secretly joins Jack at a party in third class.
>>>

chunk_id: 16392_1
doc_id: 16392
Title: Titanic 3D
Release Year: 2012
Genre: Drama
Director: James Cameron
Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates, Francis Fisher, Gloria Stuart, Bill Paxton, Bernard Hill, David Warner, Victor Garber, Jonathan Hyde, Eric Braeden, Bernard Fox, Suzy Amis, Danny Nucci, Lewis Abernathy, Nicholas Cascone
Origin/Ethnicity: American
Content (Plot):
<<<
In 1912 Southampton, 17-year-old first-class passenger Rose DeWitt Bukater, her fiancé Cal Hockley, and her mother Ruth board the luxurious Titanic. Ruth emphasizes that Rose's marriage will resolve their family's financial problems and retain their high-class persona. Distraught over the engagement, Rose considers suicide by jumping from the stern; Jack Dawson, a penniless artist, intervenes and discourages her. Discovered with Jack, Rose tells a concerned Cal that she was peering over the edge and Jack saved her from falling. When Cal becomes indifferent, she suggests to him that Jack deserves a reward. He invites Jack to dine with them in first class the following night. Jack and Rose develop a tentative friendship, despite Cal and Ruth being wary of him. Following dinner, Rose secretly joins Jack at a party in third class.
>>>

chunk_id: 6008_0
doc_id: 6008
Title: Titanic
Release Year: 1953
Genre: Biography
Director: Jean Negulesco
Cast: Barbara Stanwyck, Clifton Webb, Robert Wagner, Thelma Ritter, Richard Basehart
Origin/Ethnicity: American
Content (Plot):
<<<
At the last minute, a wealthy American expatriate in Europe, Richard Sturges (Clifton Webb), buys a steerage-class ticket (the lowest class) for the maiden voyage of the RMS Titanic from a Basque immigrant. Once aboard he seeks out his runaway wife, Julia (Barbara Stanwyck). He discovers she is trying to take their two unsuspecting children, 18-year-old Annette (Audrey Dalton) and ten-year-old Norman (Harper Carter), to her hometown of Mackinac, Michigan, to raise as down-to-earth Americans rather than rootless elitists like Richard himself.
As the ship prepares for departure, her captain, E. J. Smith (Brian Aherne), receives a hint from the shipping company representative that a record-setting speedy passage would be welcomed.
Other passengers include a wealthy woman of a working-class origin (based on a real-life Titanic survivor Molly Brown), Maude Young (Thelma Ritter); social-climbing Earl Meeker (Allyn Joslyn); a 20-year-old Purdue University tennis player, Gifford "Giff" Rogers (Robert Wagner); and George S. Healey (Richard Basehart), a Catholic priest who has been defrocked for alcoholism.
>>>

chunk_id: 19401_0
doc_id: 19401
Title: A Night to Remember
Release Year: 1958
Genre: Drama
Director: Roy Ward Baker
Cast: Kenneth More, Ronald Allen, Honor Blackman
Origin/Ethnicity: British
Content (Plot):
<<<
In 1912, the Titanic is the largest vessel afloat and is widely believed to be unsinkable. Passengers aboard for her maiden voyage are the cream of American and British society. Boarding are first class passengers Sir Richard and Lady Richard, second class passengers Mr. Clarke and Mrs. Clarke, a young newly wed couple, and steerage passengers Mr. Murphy, Mr. Gallagher and Mr. James Farrel. Second Officer Charles Lightoller is also readying for the voyage. On 10 April, Titanic sails out to sea.
On 14 April, at sea, the ship receives a number of ice warnings from other steamers. Only a few of the messages are relayed to Captain Edward J. Smith, who orders a lookout, but does not slow the ship or consider changing course.
>>>

chunk_id: 19401_2
doc_id: 19401
Title: A Night to Remember
Release Year: 1958
Genre: Drama
Director: Roy Ward Baker
Cast: Kenneth More, Ronald Allen, Honor Blackman
Origin/Ethnicity: British
Content (Plot):
<<<
On the Titanic, passengers begin to settle in for the night, while some, including Mr. Hoyle and Jay Yates stay up to gamble. Suddenly, Lookout Frederick Fleet spots an iceberg in front of the ship. The vessel turns to port but collides with the iceberg on its starboard side. Captain Smith sends for Thomas Andrews who goes to inspect the damage. Andrews determines that the ship will sink within two hours, and it lacks sufficient lifeboat capacity for everyone on board.
A distress signal is sent out, and efforts begin to signal the SS Californian, visible on the horizon 10 miles away, but its radio operator is off duty and does not hear the distress signal. Fortunately, the radio operator on the RMS Carpathia receives the distress call and alerts Captain Arthur Rostron, who orders the ship to head to the site. Unfortunately, the ship is 58 miles away, and will take around four hours to reach the Titanic. Meanwhile, the Californian remains where it is, the crew failing to comprehend why the large ship they are in sight of is firing rockets.
>>>

chunk_id: 19401_5
doc_id: 19401
Title: A Night to Remember
Release Year: 1958
Genre: Drama
Director: Roy Ward Baker
Cast: Kenneth More, Ronald Allen, Honor Blackman
Origin/Ethnicity: British
Content (Plot):
<<<
The Titanic's bow submerges, and only two collapsible lifeboats are left. Lightoller and other able seamen struggle to free them, when the ship begins its final plunge. Captain Smith remained on the Titanic‘s bridge when the forward superstructure went under, and died there. Lightoller and many others are swept off the ship. Thomas Andrews, who is asked by a steward if he will save himself, remains in the first-class smoking room, lamenting his failure to build a strong and safe ship. Passengers jump into the sea as the stern rises high into the air. The Clarkes, struggling in the water, are killed by a falling funnel. The stricken liner rapidly sinks into the icy sea.
>>>

</details>

---

### **Query 4 - Semantic Classification and Reasoning**

This question tests the system's ability to perform **high-level semantic classification** based on retrieved evidence.

Rather than asking for facts, it requires interpreting the movie's thematic structure and balancing multiple genre dimensions supported by the retrieved context.

In [102]:
ask_compare(
    "Is Titanic a romance or a historical disaster movie?",
    show_context=True
)

2026-01-17T11:00:17 | INFO | CHAT_RAG | ChatRAG started | question='Is Titanic a romance or a historical disaster movie?'
2026-01-17T11:00:17 | DEBUG | openai._base_client | Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'idempotency_key': 'stainless-python-retry-e8a7b64f-dd20-4fc5-9128-ff5bb67a4017', 'post_parser': <function Embeddings.create.<locals>.parser at 0x7cd80fafa0c0>, 'json_data': {'input': [[3957, 89862, 264, 30363, 477, 264, 13970, 21426, 5818, 30]], 'model': 'text-embedding-3-small', 'encoding_format': 'base64'}}
2026-01-17T11:00:17 | DEBUG | openai._base_client | Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2026-01-17T11:00:17 | DEBUG | httpcore.connection | close.started
2026-01-17T11:00:17 | DEBUG | httpcore.connection | close.complete
2026-01-17T11:00:17 | DEBUG | httpcore.connection | connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None


2026-01-17T11:00:17 | DEBUG | httpcore.connection | connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7cd8536afc20>
2026-01-17T11:00:17 | DEBUG | httpcore.connection | start_tls.started ssl_context=<ssl.SSLContext object at 0x7cd84c16b750> server_hostname='api.openai.com' timeout=None
2026-01-17T11:00:17 | DEBUG | httpcore.connection | start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7cd84d8426f0>
2026-01-17T11:00:17 | DEBUG | httpcore.http11 | send_request_headers.started request=<Request [b'POST']>
2026-01-17T11:00:17 | DEBUG | httpcore.http11 | send_request_headers.complete
2026-01-17T11:00:17 | DEBUG | httpcore.http11 | send_request_body.started request=<Request [b'POST']>
2026-01-17T11:00:17 | DEBUG | httpcore.http11 | send_request_body.complete
2026-01-17T11:00:17 | DEBUG | httpcore.http11 | receive_response_headers.started request=<Request [b'POST']>
2026-01-17T11:00:18 | DEBUG | httpcore.http11 | receive_response_he

## Question

Is Titanic a romance or a historical disaster movie?

<details>
<summary><b>RAG Answer</b></summary>

## Summary
The film **Titanic** can be classified as both a romance and a historical disaster movie. It intertwines a romantic narrative between the characters **Jack Dawson** and **Rose DeWitt Bukater** with the historical events surrounding the sinking of the RMS Titanic in 1912.

## Analysis
- **Romantic Elements**: The plot centers on the love story between **Jack** and **Rose**, who come from different social classes. Their relationship develops amidst the backdrop of the ship's opulence and the impending disaster, highlighting themes of love, sacrifice, and social class.
- **Historical Disaster Context**: The film is set against the real historical event of the Titanic's sinking, which adds a layer of authenticity and tragedy. The narrative includes the ship's collision with an iceberg and the subsequent chaos, emphasizing the disaster's impact on the characters and their relationships.

## Evidence
- In the 1997 version of **Titanic**, the story follows **Rose** and **Jack** as they navigate their romance while the ship is sinking, showcasing both their love and the historical tragedy of the Titanic disaster.
- The 1996 version also features a romantic plot involving **Isabella Paradine** and **Wynn Park**, set against the backdrop of the Titanic's sinking, further reinforcing the film's dual classification.

In conclusion, **Titanic** effectively combines elements of romance and historical disaster, making it a multifaceted film that appeals to a wide audience.

## Source Acknowledgment
- **Titanic** (1996)
- **Titanic** (1997)

</details>

<details>
<summary><b>LLM-only Answer (no context, no RAG prompt)</b></summary>

"Titanic," directed by James Cameron, is primarily a historical disaster film, but it also incorporates strong elements of romance. The movie is set against the backdrop of the real-life sinking of the RMS Titanic in 1912, and it depicts the tragic events surrounding the disaster. However, at its core, the film tells the love story between the characters Jack Dawson and Rose DeWitt Bukater, which serves as a central narrative thread throughout the film. This blend of historical events and a romantic storyline is one of the reasons for the film's widespread appeal and success.

</details>

<details>
<summary><b>Retrieved Context (debug)</b></summary>

chunk_id: 18117_0
doc_id: 18117
Title: Atlantic
Release Year: 1929
Genre: Drama
Director: Ewald André Dupont
Cast: Franklin Dyall, Madeleine Carroll
Origin/Ethnicity: British
Content (Plot):
<<<
Atlantic is a drama film based on the RMS Titanic and set aboard a fictional ship, called the Atlantic. The main plotline revolves around a man who has a shipboard affair with a fellow passenger, which is eventually discovered by his wife. The ship also has aboard an elderly couple, the Rools, who are on their anniversary cruise. Midway across the Atlantic Ocean, the Atlantic strikes an iceberg and is damaged to the point where it is sinking into the Atlantic. A shortage of lifeboats causes the crew to only allow women and children in (though the captain allows a few men to take to the last remaining boats as the disaster reaches its zenith) and many couples are separated. Mrs. Rool refuses to leave her husband and after the boats are gone all the passengers gather on the deck and sing "Nearer, My God, to Thee" as the Atlantic sinks into the ocean. The final scenes depict a group of passengers saying the Lord's Prayer in a flooding lounge.
>>>

chunk_id: 12857_0
doc_id: 12857
Title: Titanic
Release Year: 1996
Genre: Biography
Director: Robert Lieberman
Cast: George C. Scott, Eva Marie Saint, Peter Gallagher, Catherine Zeta-Jones
Origin/Ethnicity: American
Content (Plot):
<<<
Titanic follows three main story threads.
Isabella Paradine is traveling on the Titanic to join her husband after attending her aunt's funeral in England. On the Titanic, she meets Wynn Park, her former lover. She falls in love with him again, and after a brief affair, she sends her husband a wireless saying they cannot be together anymore (despite their daughter). When the ship starts sinking, Isabella reluctantly leaves Wynn when he forces her to board a lifeboat. As the boat is lowered, Isabella confesses a long kept secret that her daughter Claire is actually Wynn's. Later on board the RMS Carpathia she is grief-stricken when she finds Wynn's lifeless body on deck, having died of hypothermia, but luckily, when the Carpathia reaches New York she is reunited with her family who are blissfully unaware of Isabella's tryst because the telegram was never sent out due to the sinking.
>>>

chunk_id: 13153_1
doc_id: 13153
Title: Titanic
Release Year: 1997
Genre: Historical Epic, Disaster
Director: James Cameron
Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Frances Fisher, Victor Garber, Kathy Bates, Bill Paxton, Gloria Stuart, David Warner, Suzy Amis
Origin/Ethnicity: American
Content (Plot):
<<<
In 1912 Southampton, 17-year-old first-class passenger Rose DeWitt Bukater, her fiancé Cal Hockley, and her mother Ruth board the luxurious Titanic. Ruth emphasizes that Rose's marriage will resolve their family's financial problems and retain their high-class persona. Distraught over the engagement, Rose considers suicide by jumping from the stern; Jack Dawson, a penniless artist, intervenes and discourages her. Discovered with Jack, Rose tells a concerned Cal that she was peering over the edge and Jack saved her from falling. When Cal becomes indifferent, she suggests to him that Jack deserves a reward. He invites Jack to dine with them in first class the following night. Jack and Rose develop a tentative friendship, despite Cal and Ruth being wary of him. Following dinner, Rose secretly joins Jack at a party in third class.
>>>

chunk_id: 16392_1
doc_id: 16392
Title: Titanic 3D
Release Year: 2012
Genre: Drama
Director: James Cameron
Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates, Francis Fisher, Gloria Stuart, Bill Paxton, Bernard Hill, David Warner, Victor Garber, Jonathan Hyde, Eric Braeden, Bernard Fox, Suzy Amis, Danny Nucci, Lewis Abernathy, Nicholas Cascone
Origin/Ethnicity: American
Content (Plot):
<<<
In 1912 Southampton, 17-year-old first-class passenger Rose DeWitt Bukater, her fiancé Cal Hockley, and her mother Ruth board the luxurious Titanic. Ruth emphasizes that Rose's marriage will resolve their family's financial problems and retain their high-class persona. Distraught over the engagement, Rose considers suicide by jumping from the stern; Jack Dawson, a penniless artist, intervenes and discourages her. Discovered with Jack, Rose tells a concerned Cal that she was peering over the edge and Jack saved her from falling. When Cal becomes indifferent, she suggests to him that Jack deserves a reward. He invites Jack to dine with them in first class the following night. Jack and Rose develop a tentative friendship, despite Cal and Ruth being wary of him. Following dinner, Rose secretly joins Jack at a party in third class.
>>>

chunk_id: 25526_0
doc_id: 25526
Title: Lal Dupatta Malmal Ka
Release Year: 1989
Genre: Not specified
Director: Not specified
Cast: Not specified
Origin/Ethnicity: Bollywood
Content (Plot):
<<<
This is a romantic drama movie.[2]
>>>

chunk_id: 34183_0
doc_id: 34183
Title: Blood and Steel
Release Year: 2011
Genre: Not specified
Director: Not specified
Cast: Not specified
Origin/Ethnicity: Russian
Content (Plot):
<<<
The series follows the lives of the people who made the Titanic, from the workers who built it to its rich financiers. Dr. Mark Muir, an engineer and metallurgist, convinces American tycoon J.P. Morgan to hire him for the biggest shipping project in the world, the construction of the RMS Titanic at Belfast's Harland and Wolff shipyard. Mark is, in truth, a Belfast native born Marcus Malone. Now, with a new name and identity, he tries to hide his heritage from his employers, as he is Catholic and his employers, the Protestant elite that rule Belfast, dislike Catholics.
While working there, Mark falls in love with Sofia Silvestri, an Italian immigrant. However, during the construction of the Titanic, tensions rise between the lower-class workers and the rich elite. More setbacks stall the construction: Harland and Wolff want to save costs and use cheaper materials, the workers wish to form a trade union, the women suffrage movement heats up in the UK, and the pro Home Rule and pro-Unionist groups battle each other. Mark attempts to deal with these while trying to escape his past.
>>>

chunk_id: 13153_4
doc_id: 13153
Title: Titanic
Release Year: 1997
Genre: Historical Epic, Disaster
Director: James Cameron
Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Frances Fisher, Victor Garber, Kathy Bates, Bill Paxton, Gloria Stuart, David Warner, Suzy Amis
Origin/Ethnicity: American
Content (Plot):
<<<
With Rose hiding from Cal en route, the RMS Carpathia takes the survivors to New York City where Rose gives her name as Rose Dawson. Rose says she later read that Cal committed suicide after losing all his money in the Wall Street Crash of 1929.
Back in the present, Lovett decides to abandon his search after hearing Rose's story. Alone on the stern of Keldysh, Rose takes out the Heart of the Ocean — in her possession all along — and drops it into the sea over the wreck site. While she is seemingly asleep or has died in her bed,[9] photos on her dresser depict a life of freedom and adventure inspired by the life she wanted to live with Jack. A young Rose reunites with Jack at the Titanic's Grand Staircase, applauded by those who died.
>>>

chunk_id: 16392_4
doc_id: 16392
Title: Titanic 3D
Release Year: 2012
Genre: Drama
Director: James Cameron
Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates, Francis Fisher, Gloria Stuart, Bill Paxton, Bernard Hill, David Warner, Victor Garber, Jonathan Hyde, Eric Braeden, Bernard Fox, Suzy Amis, Danny Nucci, Lewis Abernathy, Nicholas Cascone
Origin/Ethnicity: American
Content (Plot):
<<<
With Rose hiding from Cal en route, the RMS Carpathia takes the survivors to New York City where Rose gives her name as Rose Dawson. Rose says she later read that Cal committed suicide after losing all his money in the Wall Street Crash of 1929.
Back in the present, Lovett decides to abandon his search after hearing Rose's story. Alone on the stern of Keldysh, Rose takes out the Heart of the Ocean — in her possession all along — and drops it into the sea over the wreck site. While she is seemingly asleep or has died in her bed,[9] photos on her dresser depict a life of freedom and adventure inspired by the life she wanted to live with Jack. A young Rose reunites with Jack at the Titanic's Grand Staircase, applauded by those who died.
>>>

chunk_id: 9758_3
doc_id: 9758
Title: Raise the Titanic
Release Year: 1980
Genre: Action
Director: Jerry Jameson
Cast: Jason Robards, Alec Guinness, Anne Archer
Origin/Ethnicity: American
Content (Plot):
<<<
Eventually, the rusting Titanic is brought to the surface by using explosives to break the hull loose from the bottom suction and compressed air tanks to fill buoyancy aids. During the ascent, the Deep Quest safely breaks away from the ship.
When Prevlov, who has been aboard a nearby Soviet spy ship, sees the Titanic, he arranges a fake distress call to draw the American naval escorts away from the operation. He then meets with Sandecker, Pitt and Seagram aboard their vessel. He tells them that his government knows all about the mineral and challenges them for both salvage of the Titanic as well as ownership of the ore, claiming it was illegally taken from Russian soil. Prevlov says that if there is to be a "superior weapon" made from the mineral, then "Russia must have it!" Sandecker tells Prevlov that they knew he was coming and what he would threaten them with. Pitt then escorts him to the deck where U.S. fighter jets and a nuclear attack submarine have arrived to protect the Titanic from their attempted piracy. Prevlov leaves in defeat.
>>>

chunk_id: 1774_0
doc_id: 1774
Title: The Richest Girl in the World
Release Year: 1934
Genre: Comedy
Director: William A. Seiter
Cast: Miriam Hopkins, Fay Wray, Joel McCrea
Origin/Ethnicity: American
Content (Plot):
<<<
When the Titanic sinks, infant Dorothy Hunter (Miriam Hopkins) is left an orphan. She is brought up by John Connors (Henry Stephenson), whose wife was also lost in the disaster. He goes to such great lengths to protect her privacy that, though she has grown into adulthood and acquired the title of the richest girl in the world, the newspapers do not have an up-to-date photograph of her. She returns to America, but her friend and secretary, Sylvia Lockwood (Fay Wray), impersonates her in a meeting with the managers of her fortune.
After seeing how happy Sylvia is with her new husband, Phillip (Reginald Denny), she broaches the topic of setting a wedding date with Donald (George Meeker), her longtime fiancé. He is forced to admit that he has fallen in love with someone else and was getting up the nerve to tell her. Since she is not the least bit in love, she congratulates him. However, it is too late to cancel the party in which she had planned to announce their wedding.
>>>

</details>

---

### **Query 5 - Open-Ended Genre and Temporal Aggregation**

This query evaluates retrieval performance in **open-ended aggregation tasks**, requiring the system to identify multiple relevant movies based on genre and release period.

It highlights common failure modes such as hallucinated examples or weak temporal filtering.

In [103]:
ask_compare(
    "List some science fiction movies from the 1990s.", 
    show_context=True
)

2026-01-17T11:00:29 | INFO | CHAT_RAG | ChatRAG started | question='List some science fiction movies from the 1990s.'
2026-01-17T11:00:29 | DEBUG | openai._base_client | Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'idempotency_key': 'stainless-python-retry-1880beb9-4609-4b70-aa3f-5741d576192f', 'post_parser': <function Embeddings.create.<locals>.parser at 0x7cd80fb080e0>, 'json_data': {'input': [[861, 1063, 8198, 17422, 9698, 505, 279, 220, 2550, 15, 82, 13]], 'model': 'text-embedding-3-small', 'encoding_format': 'base64'}}
2026-01-17T11:00:29 | DEBUG | openai._base_client | Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2026-01-17T11:00:29 | DEBUG | httpcore.connection | close.started
2026-01-17T11:00:29 | DEBUG | httpcore.connection | close.complete
2026-01-17T11:00:29 | DEBUG | httpcore.connection | connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None


2026-01-17T11:00:29 | DEBUG | httpcore.connection | connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7cd84d852a50>
2026-01-17T11:00:29 | DEBUG | httpcore.connection | start_tls.started ssl_context=<ssl.SSLContext object at 0x7cd84c16b750> server_hostname='api.openai.com' timeout=None
2026-01-17T11:00:29 | DEBUG | httpcore.connection | start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7cd84d8504a0>
2026-01-17T11:00:29 | DEBUG | httpcore.http11 | send_request_headers.started request=<Request [b'POST']>
2026-01-17T11:00:29 | DEBUG | httpcore.http11 | send_request_headers.complete
2026-01-17T11:00:29 | DEBUG | httpcore.http11 | send_request_body.started request=<Request [b'POST']>
2026-01-17T11:00:29 | DEBUG | httpcore.http11 | send_request_body.complete
2026-01-17T11:00:29 | DEBUG | httpcore.http11 | receive_response_headers.started request=<Request [b'POST']>
2026-01-17T11:00:30 | DEBUG | httpcore.http11 | receive_response_he

## Question

List some science fiction movies from the 1990s.

<details>
<summary><b>RAG Answer</b></summary>

No sufficient information was found in the retrieved movie documents to answer this question. 

### Source Acknowledgment
- **Creepozoids** (1987)
- **Meet Dave** (2008)
- **Impostor** (2002)
- **Spacehunter: Adventures in the Forbidden Zone** (1983)
- **Queen of Blood** (1966)
- **Prometheus** (2012)
- **Remote Control** (1988)
- **Time Travelers** (1964)
- **Galaxy Turnpike** (2015)

</details>

<details>
<summary><b>LLM-only Answer (no context, no RAG prompt)</b></summary>

Here are some notable science fiction movies from the 1990s:

1. **Total Recall** (1990) - Directed by Paul Verhoeven, this film stars Arnold Schwarzenegger as a man who discovers his memories may not be real.
2. **Terminator 2: Judgment Day** (1991) - James Cameron's sequel to the original "Terminator," featuring advanced special effects and a compelling story about time travel and artificial intelligence.
3. **The Matrix** (1999) - Directed by the Wachowskis, this groundbreaking film explores themes of reality and artificial intelligence, featuring Keanu Reeves as Neo.
4. **Jurassic Park** (1993) - Directed by Steven Spielberg, this film brought dinosaurs to life using groundbreaking CGI and tells the story of a theme park filled with cloned dinosaurs.
5. **Starship Troopers** (1997) - Another film by Paul Verhoeven, this satirical take on militarism and fascism features humans battling giant alien bugs.
6. **Gattaca** (1997) - A thought-provoking film about genetic engineering and the implications of a society that values genetic perfection.
7. **The Fifth Element** (1997) - Directed by Luc Besson, this visually stunning film combines action, humor, and a unique vision of the future.
8. **Contact** (1997) - Based on Carl Sagan's novel, this film explores the search for extraterrestrial life and the implications of making contact.
9. **The X-Files: Fight the Future** (1998) - A feature film based on the popular TV series, continuing the story of FBI agents Mulder and Scully as they investigate alien conspiracies.
10. **Dark City** (1998) - A neo-noir science fiction film that explores themes of identity and reality in a city controlled by mysterious beings.

These films reflect a variety of themes and styles within the science fiction genre during the 1990s.

</details>

<details>
<summary><b>Retrieved Context (debug)</b></summary>

chunk_id: 10686_0
doc_id: 10686
Title: Creepozoids
Release Year: 1987
Genre: Sci-Fi
Director: David DeCoteau
Cast: Linnea Quigley
Origin/Ethnicity: American
Content (Plot):
<<<
Set in 1998, six years after an apocalyptic nuclear war, a group of five U.S. Army deserters take refuge from acid rain in a seemingly abandoned laboratory complex in the ruins of Los Angeles. They soon discover that the lab was a top-secret government research center, where a hideous (if rather fickle) genetically-engineered monster still lurks.
>>>

chunk_id: 15480_0
doc_id: 15480
Title: Meet Dave
Release Year: 2008
Genre: Comedy
Director: Brian Robbins
Cast: Eddie Murphy, Gabrielle Union, Elizabeth Banks, Ed Helms
Origin/Ethnicity: American
Content (Plot):
<<<
In his New York City apartment, a young boy named Josh Morrison (Austyn Lind Myers) stares through his telescope at an object falling from the sky. It is a golf-ball-sized metal ball which flies through the window and lands in his fishbowl, quickly draining the water along with the goldfish. He decides to show it at his school's science class presentation.
Some months later a massive fireball crashes into the water near Liberty Island. It is revealed to be a spaceship which resembles a human (Eddie Murphy), controlled by 100 tiny humanoid aliens. Its Captain (also played by Murphy) pilots the spaceship from the command deck located in its head, with the help of his second-in-command Number 2 (Ed Helms), and researcher Number 3 (Gabrielle Union). The spaceship looks very human, and displays numerous superpowers, but the aliens don't know how to make the "ship" act like a human. A superstitious cop named Dooley (Scott Caan) desperately searches for the alien.
>>>

chunk_id: 14076_0
doc_id: 14076
Title: Impostor
Release Year: 2002
Genre: Science Fiction
Director: Gary Fleder
Cast: Gary Sinise, Madeleine Stowe, Mekhi Phifer, Vincent D'Onofrio
Origin/Ethnicity: American
Content (Plot):
<<<
The film takes place in the year 2079. Forty-five years earlier, Earth was attacked by a hostile and implacable alien civilization from Alpha Centauri. Force shield domes are put in place to protect cities, and a totalitarian global military government is established to effect the war and the survival of humans. The Centaurians have never been physically seen.
The film follows Spencer Olham, a designer of top-secret government weapons. He is arrested by Major Hathaway of the Earth Security Administration (ESA), being identified as a replicant created by the aliens. The ESA intercepted an alien transmission which cryptanalysts decoded as programming Olham's target to be the Chancellor, whom he was scheduled to meet. Such replicants are perfect biological copies of existing humans, complete with transplanted memories, and do not know they are replicants. Each has a powerful "u-bomb" in their chest, which can only be detected by dissection or a high-tech medical scan, since it only assembles itself when it gets in proximity to its target. Detection via the special scan works by comparing against a previous scan, if there was one.
>>>

chunk_id: 10133_0
doc_id: 10133
Title: Spacehunter: Adventures in the Forbidden Zone
Release Year: 1983
Genre: Sci-Fi
Director: Lamont Johnson
Cast: Peter Strauss, Molly Ringwald, Ernie Hudson
Origin/Ethnicity: American
Content (Plot):
<<<
Set in the early 22nd century, the film opens with the destruction of a space cruise liner that becomes irreparably damaged and destroyed by a bolt of nebular lightning. The only apparent survivors are three beautiful women – Nova (Cali Timmins), Reena (Aleisa Shirley), and Meagan (Deborah Pratt) – who get away in an escape pod and land on the nearest habitable planet. There, they are quickly accosted by the hostile natives and taken aboard a sail-driven vehicle resembling a pirate ship on rails.
In space, an alert goes out for the safe return of the women with a reward of 3,000 "mega-credits". A small-time salvage operator named Wolff (Peter Strauss) intercepts the message and heads to the planet. Joining him is his female engineer Chalmers (Andrea Marcovicci), who learns the planet – called Terra XI – is a failed colony that fell victim to a deadly plague and civil warfare. Wolff risks the dangers believing the reward will solve his debt problems.
>>>

chunk_id: 8134_0
doc_id: 8134
Title: Queen of Blood
Release Year: 1966
Genre: Sci-Fi
Director: Curtis Harrington
Cast: John Saxon, Dennis Hopper
Origin/Ethnicity: American
Content (Plot):
<<<
The year is 1990. Space travel is well-established since humans first landed on the Moon twenty years earlier. At the International Institute of Space Technology, communications expert and astronaut Laura James monitors strange signals being received from outer space. Laura's superior, Dr. Farraday, translates the signal and discovers that it is from an alien race, who are sending an ambassador to Earth. Soon after, however, Laura receives a video log showing that the aliens' starship has crash-landed on Mars.
>>>

chunk_id: 16351_0
doc_id: 16351
Title: Prometheus
Release Year: 2012
Genre: Action, Horror, Science Fiction
Director: Ridley Scott
Cast: Noomi Rapace, Michael Fassbender, Guy Pearce, Idris Elba, Logan Marshall-Green, Charlize Theron
Origin/Ethnicity: American
Content (Plot):
<<<
As a spacecraft departs a planet, a humanoid alien drinks an iridescent liquid and then dissolves. The remains of the alien cascade into a waterfall. The alien's DNA strands mix with the water.
In 2089, archaeologists Elizabeth Shaw and Charlie Holloway discover a star map in Scotland that matches others from several unconnected ancient cultures. They interpret this as an invitation from humanity's forerunners, the "Engineers". Peter Weyland, the elderly CEO of Weyland Corporation, funds an expedition, aboard the scientific vessel Prometheus, to follow the map to the distant moon LV-223. The ship's crew travels in stasis while the android David monitors their voyage. Arriving in December 2093, mission-director Meredith Vickers informs them of their mission to find the Engineers and not to make contact without her permission.
>>>

chunk_id: 21236_0
doc_id: 21236
Title: Prometheus
Release Year: 2012
Genre: Not specified
Director: Director: Ridley Scott
Cast: Director: Ridley Scott, Cast: Noomi Rapace, Michael Fassbender, Guy Pearce, Idris Elba, Logan Marshall-Green, Charlize Theron
Origin/Ethnicity: British
Content (Plot):
<<<
As a spacecraft departs a planet, a humanoid alien drinks an iridescent liquid and then dissolves. The remains of the alien cascade into a waterfall. The alien's DNA strands mix with the water.
In 2089, archaeologists Elizabeth Shaw and Charlie Holloway discover a star map in Scotland that matches others from several unconnected ancient cultures. They interpret this as an invitation from humanity's forerunners, the "Engineers". Peter Weyland, the elderly CEO of Weyland Corporation, funds an expedition, aboard the scientific vessel Prometheus, to follow the map to the distant moon LV-223. The ship's crew travels in stasis while the android David monitors their voyage. Arriving in December 2093, mission-director Meredith Vickers informs them of their mission to find the Engineers and not to make contact without her permission.
>>>

chunk_id: 11051_0
doc_id: 11051
Title: Remote Control
Release Year: 1988
Genre: Comedy
Director: Jeff Lieberman
Cast: Kevin Dillon, Jennifer Tilly
Origin/Ethnicity: American
Content (Plot):
<<<
A video store clerk stumbles onto an alien plot to take over earth by brainwashing people with a bad '50s science fiction movie. He and his friends race to stop the aliens before the tapes can be distributed world-wide.
>>>

chunk_id: 7926_1
doc_id: 7926
Title: Time Travelers
Release Year: 1964
Genre: Sci-Fi
Director: Ib Melchior
Cast: Preston Foster, Merry Anders
Origin/Ethnicity: American
Content (Plot):
<<<
The year is 2071 A.D. City leader Dr. Varno (John Hoyt) explains that Earth is unable to support life. The residents are frantically working on a spacecraft that will take them to a planet orbiting Alpha Centauri. The four time travelers, told they may not join that space voyage, are allowed to work on recreating their time portal to return to their time. Before the colony residents can lift off, the degenerate mutant humans break in and destroy the ship and encroach on the city.
Dr. Varno determines that now the only hope is the time portal, so he commits the city's remaining resouces to help von Steiner, Connors, White and McKee rebuild the time portal with future technology. As they feverishly work, the mutants are invading the colony; along with a few people from the future, the four travelers escape back to the present just ahead of the mutants. One person throws an object back through the gateway that damages the equipment on the other side and shuts down the portal.
>>>

chunk_id: 33832_0
doc_id: 33832
Title: Galaxy Turnpike
Release Year: 2015
Genre: Science Fiction, Comedy
Director: Kōki Mitani
Cast: Shingo Katori, Haruka Ayase, Shun Oguri
Origin/Ethnicity: Japanese
Content (Plot):
<<<
The film is set in 2265.[3]
>>>

</details>

---

### **Query 6 - Explicit Genre-Constrained Retrieval**

By explicitly stating the genre, this query reduces interpretive ambiguity and isolates the retriever's ability to leverage **genre-related signals and metadata**.

The goal is to assess whether explicit constraints lead to more grounded and diverse results compared to broader genre prompts.

In [104]:
ask_compare(
    "List some science fiction genre movies released in the 1990s.",
    show_context=True
)

2026-01-17T11:00:43 | INFO | CHAT_RAG | ChatRAG started | question='List some science fiction genre movies released in the 1990s.'
2026-01-17T11:00:43 | DEBUG | openai._base_client | Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'idempotency_key': 'stainless-python-retry-debca50a-a131-4589-ab40-d6d9fff524b5', 'post_parser': <function Embeddings.create.<locals>.parser at 0x7cd80fb08220>, 'json_data': {'input': [[861, 1063, 8198, 17422, 17779, 9698, 6004, 304, 279, 220, 2550, 15, 82, 13]], 'model': 'text-embedding-3-small', 'encoding_format': 'base64'}}
2026-01-17T11:00:43 | DEBUG | openai._base_client | Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2026-01-17T11:00:43 | DEBUG | httpcore.connection | close.started
2026-01-17T11:00:43 | DEBUG | httpcore.connection | close.complete
2026-01-17T11:00:43 | DEBUG | httpcore.connection | connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
2026-01-17T

## Question

List some science fiction genre movies released in the 1990s.

<details>
<summary><b>RAG Answer</b></summary>

## Summary
Based on the retrieved context, the following science fiction genre movies were released in the 1990s:

- **Queen of Blood** (1966) - Although this film is set in 1990, it was released earlier and does not fit the 1990s release criteria.
- **Reptilian** (1999) - A kaiju film involving an archaeological expedition and an alien mothership.
- **Yonggary** (1999) - Similar to "Reptilian," it involves an archaeological dig and an alien presence.

## Analysis
The only two films explicitly mentioned that were released in the 1990s are **Reptilian** and **Yonggary**, both from 1999. **Queen of Blood**, while set in 1990, was released in 1966 and does not qualify as a 1990s release.

## Evidence
- **Reptilian**: "In Southeast Asian, an archaeological party explore some caverns underground... Two years later, an alien mothership arrives near Earth’s orbit..." (1999)
- **Yonggary**: "In Southeast Asian, an archaeological party explore some caverns underground... Two years later, an alien mothership arrives near Earth’s orbit..." (1999)
- **Queen of Blood**: "The year is 1990... communications expert and astronaut Laura James monitors strange signals being received from outer space." (1966)

No sufficient information was found in the retrieved movie documents to provide additional titles from the 1990s.

## Source Acknowledgment
- Queen of Blood
- Reptilian
- Yonggary

</details>

<details>
<summary><b>LLM-only Answer (no context, no RAG prompt)</b></summary>

Here are some notable science fiction movies released in the 1990s:

1. **Total Recall** (1990)
2. **Terminator 2: Judgment Day** (1991)
3. **The Matrix** (1999)
4. **Jurassic Park** (1993)
5. **Starship Troopers** (1997)
6. **Gattaca** (1997)
7. **The Fifth Element** (1997)
8. **Contact** (1997)
9. **Blade Runner: The Final Cut** (1992) - re-release of the original 1982 film
10. **The X-Files** (1998)
11. **Dark City** (1998)
12. **The Iron Giant** (1999)
13. **Star Wars: Episode I - The Phantom Menace** (1999)
14. **Event Horizon** (1997)
15. **The Abyss** (1990)

These films represent a variety of themes and styles within the science fiction genre during that decade.

</details>

<details>
<summary><b>Retrieved Context (debug)</b></summary>

chunk_id: 10686_0
doc_id: 10686
Title: Creepozoids
Release Year: 1987
Genre: Sci-Fi
Director: David DeCoteau
Cast: Linnea Quigley
Origin/Ethnicity: American
Content (Plot):
<<<
Set in 1998, six years after an apocalyptic nuclear war, a group of five U.S. Army deserters take refuge from acid rain in a seemingly abandoned laboratory complex in the ruins of Los Angeles. They soon discover that the lab was a top-secret government research center, where a hideous (if rather fickle) genetically-engineered monster still lurks.
>>>

chunk_id: 33832_0
doc_id: 33832
Title: Galaxy Turnpike
Release Year: 2015
Genre: Science Fiction, Comedy
Director: Kōki Mitani
Cast: Shingo Katori, Haruka Ayase, Shun Oguri
Origin/Ethnicity: Japanese
Content (Plot):
<<<
The film is set in 2265.[3]
>>>

chunk_id: 15480_0
doc_id: 15480
Title: Meet Dave
Release Year: 2008
Genre: Comedy
Director: Brian Robbins
Cast: Eddie Murphy, Gabrielle Union, Elizabeth Banks, Ed Helms
Origin/Ethnicity: American
Content (Plot):
<<<
In his New York City apartment, a young boy named Josh Morrison (Austyn Lind Myers) stares through his telescope at an object falling from the sky. It is a golf-ball-sized metal ball which flies through the window and lands in his fishbowl, quickly draining the water along with the goldfish. He decides to show it at his school's science class presentation.
Some months later a massive fireball crashes into the water near Liberty Island. It is revealed to be a spaceship which resembles a human (Eddie Murphy), controlled by 100 tiny humanoid aliens. Its Captain (also played by Murphy) pilots the spaceship from the command deck located in its head, with the help of his second-in-command Number 2 (Ed Helms), and researcher Number 3 (Gabrielle Union). The spaceship looks very human, and displays numerous superpowers, but the aliens don't know how to make the "ship" act like a human. A superstitious cop named Dooley (Scott Caan) desperately searches for the alien.
>>>

chunk_id: 8134_0
doc_id: 8134
Title: Queen of Blood
Release Year: 1966
Genre: Sci-Fi
Director: Curtis Harrington
Cast: John Saxon, Dennis Hopper
Origin/Ethnicity: American
Content (Plot):
<<<
The year is 1990. Space travel is well-established since humans first landed on the Moon twenty years earlier. At the International Institute of Space Technology, communications expert and astronaut Laura James monitors strange signals being received from outer space. Laura's superior, Dr. Farraday, translates the signal and discovers that it is from an alien race, who are sending an ambassador to Earth. Soon after, however, Laura receives a video log showing that the aliens' starship has crash-landed on Mars.
>>>

chunk_id: 10133_0
doc_id: 10133
Title: Spacehunter: Adventures in the Forbidden Zone
Release Year: 1983
Genre: Sci-Fi
Director: Lamont Johnson
Cast: Peter Strauss, Molly Ringwald, Ernie Hudson
Origin/Ethnicity: American
Content (Plot):
<<<
Set in the early 22nd century, the film opens with the destruction of a space cruise liner that becomes irreparably damaged and destroyed by a bolt of nebular lightning. The only apparent survivors are three beautiful women – Nova (Cali Timmins), Reena (Aleisa Shirley), and Meagan (Deborah Pratt) – who get away in an escape pod and land on the nearest habitable planet. There, they are quickly accosted by the hostile natives and taken aboard a sail-driven vehicle resembling a pirate ship on rails.
In space, an alert goes out for the safe return of the women with a reward of 3,000 "mega-credits". A small-time salvage operator named Wolff (Peter Strauss) intercepts the message and heads to the planet. Joining him is his female engineer Chalmers (Andrea Marcovicci), who learns the planet – called Terra XI – is a failed colony that fell victim to a deadly plague and civil warfare. Wolff risks the dangers believing the reward will solve his debt problems.
>>>

chunk_id: 34415_0
doc_id: 34415
Title: Reptilian
Release Year: 1999
Genre: Kaiju
Director: Shim Hyung-rae
Cast: Dan Cashman
Origin/Ethnicity: South_Korean
Content (Plot):
<<<
Based on the 2001 version. The 1999 version has never been released on home video on any market.
In Southeast Asian, an archaeological party explore some caverns underground. Dr. Campbell and Dr. Hughes are the two leaders of the archaeological expedition, and get separated. While Dr. Hughes finds an alien corpse with a fossilized diamond, Campbell finds hieroglyphics at the cost of the party except for Hughes and himself. Two years later, an alien mothership arrives near Earth’s orbit, which destroys two American satellites that gets the attention of Parker, who reports it to General Murdock of the United National Defense Agency (UNDA).
Bud Black, a cameraman, learns from a colleague about a dinosaur dig, led by Campbell along with his assistant Holly. Dr. Hughes, who has been believed to be dead for the last two years, shows up at the dig site to warn Campbell but is quickly removed from the site. Subsequently, the alien ship sends beams to reanimate the dinosaur but at the cost of some of the diggers’ lives.
>>>

chunk_id: 34420_0
doc_id: 34420
Title: Yonggary
Release Year: 1999
Genre: Not specified
Director: Shim Hyung-rae
Cast: Not specified
Origin/Ethnicity: South_Korean
Content (Plot):
<<<
Based on the 2001 version. The 1999 version has never been released on home video on any market.
In Southeast Asian, an archaeological party explore some caverns underground. Dr. Campbell and Dr. Hughes are the two leaders of the archaeological expedition, and get separated. While Dr. Hughes finds an alien corpse with a fossilized diamond, Campbell finds hieroglyphics at the cost of the party except for Hughes and himself. Two years later, an alien mothership arrives near Earth’s orbit, which destroys two American satellites that gets the attention of Parker, who reports it to General Murdock of the United National Defense Agency (UNDA).
Bud Black, a cameraman, learns from a colleague about a dinosaur dig, led by Campbell along with his assistant Holly. Dr. Hughes, who has been believed to be dead for the last two years, shows up at the dig site to warn Campbell but is quickly removed from the site. Subsequently, the alien ship sends beams to reanimate the dinosaur but at the cost of some of the diggers’ lives.
>>>

chunk_id: 14076_0
doc_id: 14076
Title: Impostor
Release Year: 2002
Genre: Science Fiction
Director: Gary Fleder
Cast: Gary Sinise, Madeleine Stowe, Mekhi Phifer, Vincent D'Onofrio
Origin/Ethnicity: American
Content (Plot):
<<<
The film takes place in the year 2079. Forty-five years earlier, Earth was attacked by a hostile and implacable alien civilization from Alpha Centauri. Force shield domes are put in place to protect cities, and a totalitarian global military government is established to effect the war and the survival of humans. The Centaurians have never been physically seen.
The film follows Spencer Olham, a designer of top-secret government weapons. He is arrested by Major Hathaway of the Earth Security Administration (ESA), being identified as a replicant created by the aliens. The ESA intercepted an alien transmission which cryptanalysts decoded as programming Olham's target to be the Chancellor, whom he was scheduled to meet. Such replicants are perfect biological copies of existing humans, complete with transplanted memories, and do not know they are replicants. Each has a powerful "u-bomb" in their chest, which can only be detected by dissection or a high-tech medical scan, since it only assembles itself when it gets in proximity to its target. Detection via the special scan works by comparing against a previous scan, if there was one.
>>>

chunk_id: 7926_1
doc_id: 7926
Title: Time Travelers
Release Year: 1964
Genre: Sci-Fi
Director: Ib Melchior
Cast: Preston Foster, Merry Anders
Origin/Ethnicity: American
Content (Plot):
<<<
The year is 2071 A.D. City leader Dr. Varno (John Hoyt) explains that Earth is unable to support life. The residents are frantically working on a spacecraft that will take them to a planet orbiting Alpha Centauri. The four time travelers, told they may not join that space voyage, are allowed to work on recreating their time portal to return to their time. Before the colony residents can lift off, the degenerate mutant humans break in and destroy the ship and encroach on the city.
Dr. Varno determines that now the only hope is the time portal, so he commits the city's remaining resouces to help von Steiner, Connors, White and McKee rebuild the time portal with future technology. As they feverishly work, the mutants are invading the colony; along with a few people from the future, the four travelers escape back to the present just ahead of the mutants. One person throws an object back through the gateway that damages the equipment on the other side and shuts down the portal.
>>>

chunk_id: 10580_0
doc_id: 10580
Title: Night of the Creeps
Release Year: 1986
Genre: Horror Comedy
Director: Fred Dekker
Cast: Jason Lively, Tom Atkins
Origin/Ethnicity: American
Content (Plot):
<<<
In 1959, on board a spacecraft, two aliens race to keep an experiment from being released by a third member of the crew. The seemingly possessed third alien shoots the canister into space where it crashes to Earth. Nearby, a college man takes his date to a parking spot when they see a falling star and investigate. It lands in the path of an escaped criminally insane mental patient. As his date is attacked by the axe-wielding maniac, the boy finds the canister, from which a small slug-like thing jumps out and into his mouth.
Twenty-seven years later in 1986, Chris Romero pines over a love lost, supported by his disabled friend J.C. During pledge week at Corman University, Chris spots a girl, Cynthia Cronenberg, and falls instantly in love. To get her attention, he decides to join a fraternity. Cynthia's boyfriend, who heads the Beta Epsilon fraternity, tasks them with stealing a cadaver from the university medical center and depositing it on the steps of a sorority house. Chris and J.C. find a frozen corpse in a secret room, but when it grabs them, they flee. The thawed corpse then kills a medical student working at the lab.
>>>

</details>

---

### **Query - Metadata-Driven Retrieval by Country**

This query focuses on **metadata-based retrieval**, testing whether country-of-production information was preserved, embedded, and retrievable.

It also exposes whether the system relies on grounded evidence or falls back to stereotypes or prior knowledge in the absence of explicit retrieval.

In [105]:
ask_compare("Which movies were made in India?", show_context=True)

2026-01-17T11:00:57 | INFO | CHAT_RAG | ChatRAG started | question='Which movies were made in India?'
2026-01-17T11:00:57 | DEBUG | openai._base_client | Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'idempotency_key': 'stainless-python-retry-7642341d-e426-4a58-bd0e-0b452dff713a', 'post_parser': <function Embeddings.create.<locals>.parser at 0x7cd80fb08c20>, 'json_data': {'input': [[23956, 9698, 1051, 1903, 304, 6890, 30]], 'model': 'text-embedding-3-small', 'encoding_format': 'base64'}}
2026-01-17T11:00:57 | DEBUG | openai._base_client | Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2026-01-17T11:00:57 | DEBUG | httpcore.connection | close.started
2026-01-17T11:00:57 | DEBUG | httpcore.connection | close.complete
2026-01-17T11:00:57 | DEBUG | httpcore.connection | connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None


2026-01-17T11:00:58 | DEBUG | httpcore.connection | connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7cd84d574380>
2026-01-17T11:00:58 | DEBUG | httpcore.connection | start_tls.started ssl_context=<ssl.SSLContext object at 0x7cd84c16b750> server_hostname='api.openai.com' timeout=None
2026-01-17T11:00:58 | DEBUG | httpcore.connection | start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7cd84d853aa0>
2026-01-17T11:00:58 | DEBUG | httpcore.http11 | send_request_headers.started request=<Request [b'POST']>
2026-01-17T11:00:58 | DEBUG | httpcore.http11 | send_request_headers.complete
2026-01-17T11:00:58 | DEBUG | httpcore.http11 | send_request_body.started request=<Request [b'POST']>
2026-01-17T11:00:58 | DEBUG | httpcore.http11 | send_request_body.complete
2026-01-17T11:00:58 | DEBUG | httpcore.http11 | receive_response_headers.started request=<Request [b'POST']>
2026-01-17T11:00:58 | DEBUG | httpcore.http11 | receive_response_he

## Question

Which movies were made in India?

<details>
<summary><b>RAG Answer</b></summary>

## Summary
The retrieved context provides information about several movies made in India, spanning various genres and themes. The films include **Samar** (1999), **Filmistaan** (2014), **Touring Talkies** (2013), **Madras to Pondicherry** (1966), **Aridhu Aridhu** (2010), **Chennai 2 Singapore** (2017), **Kunthapura** (2013), **Cool...Sakkath Hot Maga** (2011), **Neelagiri Express** (1968), and **Mere Dost Picture Abhi Baki Hai** (2012).

## Analysis
The films listed reflect a diverse range of storytelling, from dramas addressing social issues like caste and terrorism to comedies exploring the film industry and romantic narratives. Each film is rooted in its cultural context, showcasing the richness of Indian cinema across different languages and regions.

## Evidence
- **Samar** (1999): A drama centered on India's caste system and community conflicts over a water pump installation.
- **Filmistaan** (2014): A comedy about an Indian Bollywood fan who is kidnapped and finds common ground with his captors through cinema.
- **Touring Talkies** (2013): A drama about the tradition of roaming cinema in Maharashtra.
- **Madras to Pondicherry** (1966): A road movie featuring a young woman escaping gangsters while traveling by bus.
- **Aridhu Aridhu** (2010): A film addressing global terrorism and violence.
- **Chennai 2 Singapore** (2017): A romantic comedy about a filmmaker's misadventures in Singapore.
- **Kunthapura** (2013): A period drama set in colonial India, focusing on freedom fighters and local peasants.
- **Cool...Sakkath Hot Maga** (2011): A romantic comedy featuring a college student.
- **Neelagiri Express** (1968): A murder mystery set on a moving train.
- **Mere Dost Picture Abhi Baki Hai** (2012): A comedic insight into the Bollywood film industry.

## Source Acknowledgment
- Samar (1999)
- Filmistaan (2014)
- Touring Talkies (2013)
- Madras to Pondicherry (1966)
- Aridhu Aridhu (2010)
- Chennai 2 Singapore (2017)
- Kunthapura (2013)
- Cool...Sakkath Hot Maga (2011)
- Neelagiri Express (1968)
- Mere Dost Picture Abhi Baki Hai (2012)

</details>

<details>
<summary><b>LLM-only Answer (no context, no RAG prompt)</b></summary>

India has a rich and diverse film industry, producing movies in various languages and genres. Here are some notable films made in India across different languages:

### Hindi Cinema (Bollywood)
1. **Sholay** (1975) - A classic action-adventure film.
2. **Dilwale Dulhania Le Jayenge** (1995) - A romantic drama that became a cultural phenomenon.
3. **Lagaan** (2001) - A historical sports drama nominated for an Academy Award.
4. **3 Idiots** (2009) - A comedy-drama about the Indian education system.
5. **Gully Boy** (2019) - A musical drama inspired by the lives of street rappers in Mumbai.

### Tamil Cinema
1. **Baahubali: The Beginning** (2015) - An epic action film that gained international acclaim.
2. **Enthiran (Robot)** (2010) - A science fiction film featuring Rajinikanth.
3. **Super Deluxe** (2019) - A critically acclaimed film with multiple intertwining stories.

### Telugu Cinema
1. **Magadheera** (2009) - A fantasy action film that was a major box office success.
2. **Eega** (2012) - A unique fantasy film about a man reincarnated as a housefly.
3. **RRR** (2022) - A historical fiction film that received global recognition.

### Bengali Cinema
1. **Pather Panchali** (1955) - A landmark film by Satyajit Ray, known for its realism.
2. **Charulata** (1964) - Another classic by Satyajit Ray, exploring themes of loneliness and desire.
3. **Chokher Bali** (2003) - A film based on Rabindranath Tagore's novel.

### Malayalam Cinema
1. **Drishyam** (2013) - A thriller that became a huge hit and was remade in several languages.
2. **Kumbalangi Nights** (2019) - A critically acclaimed film about brotherhood and relationships.
3. **Bangalore Days** (2014) - A coming-of-age film that resonated with younger audiences.

### Kannada Cinema
1. **KGF: Chapter 1** (2018) - A blockbuster action film that gained pan-Indian popularity.
2. **Ugramm** (2014) - A film that revitalized the Kannada action genre.
3. **Shivaaji Surathkal** (2019) - A mystery thriller that received positive reviews.

### Other Regional Cinemas
1. **Sardar Udham** (2021) - A Hindi film based on the life of revolutionary Udham Singh.
2. **The Disciple** (2020) - A Hindi film exploring the world of Indian classical music.
3. **Jersey** (2021) - A Hindi remake of a Telugu film about cricket and redemption.

This is just a small selection, and there are many more films across various languages and genres that showcase the diversity and creativity of Indian cinema.

</details>

<details>
<summary><b>Retrieved Context (debug)</b></summary>

chunk_id: 26009_0
doc_id: 26009
Title: Samar
Release Year: 1999
Genre: Drama
Director: Shyam Benegal
Cast: Seema Biswas, Ravi Jhankal, Rajeshwari Sachdev
Origin/Ethnicity: Bollywood
Content (Plot):
<<<
The larger theme of the film is centred on India's caste system, though it is depicted as a film within a film.
In a small village in Madhya Pradesh, two different communities fight over a water pump installation. When a member of one of the communities, Nathu (Kishore Kadam) decides to protest against a decision he feels is unjust, he angers the local land owner, who decides to impose economic sanctions on the community in an effort to starve them out of the village. When Nathu's house is burned down under mysterious circumstances, Nathu seeks the comfort of a temple, and prays for a solution. Instead he finds himself abused and beaten by the land owner for breaking a rule that bans members of Nathu's community from entering the temple. It later emerges that the situation in the area is being used as a plot for a film made in Bombay, however characters featured in the film are misrepresented, which leads to tension on the set and eventually violence erupts.
>>>

chunk_id: 26992_0
doc_id: 26992
Title: Filmistaan
Release Year: 2014
Genre: Comedy
Director: Nitin Kakkar
Cast: Sharib Hashmi, Inaamulhaq, Kumud Mishra, Gopal Dutt
Origin/Ethnicity: Bollywood
Content (Plot):
<<<
In Mumbai, an affable Indian Punjabi Hindu Bollywood buff and wanna-be-actor Suneet ("Sunny"), who works as an assistant director, fantasizes on becoming a heart-throb star. However, at every audition he is summarily thrown out. Undeterred, he goes with an American crew to remote areas in Rajasthan to work on a documentary.
One day, a terrorist, Muslim Jihadist group kidnaps him instead of an American crew-member. Sunny finds himself on enemy border amidst guns and pathani-clad guards, who decide to keep him hostage until they locate their original target. The house where he is confined belongs to a Pakistani Muslim, whose trade stems from pirated Hindi films, which he brings back every time he crosses the border. Soon, the two factions realize that they share a human and cultural bond. The film shows how cinema can be the universal panacea for co-existence.
>>>

chunk_id: 28711_0
doc_id: 28711
Title: Touring Talkies
Release Year: 2013
Genre: Drama
Director: Gajendra Ahire
Cast: Subodh Bhave, Trupti Bhoir, Kishor Kadam, Suhas Palshikar, Milind Shinde, Vaibhav Mangale, Neha Pendse, Chinmay Sant
Origin/Ethnicity: Marathi
Content (Plot):
<<<
The movie is based on the concept of roaming cinema which was one of the oldest traditions in the bygone era of Indian cinema where the movies were showcased in tents for the local folks also known as Touring talkies. For the past few decades these films have made a common recurring occurrences in most of the Carnival Fun fair taking place across villages where the Marathi Cinema is adored in a completely distinctive fashion
>>>

chunk_id: 29354_0
doc_id: 29354
Title: Madras to Pondicherry
Release Year: 1966
Genre: Not specified
Director: Thirumalai Mahalingam
Cast: Nagesh, Ravichandran, A. Karunanidhi
Origin/Ethnicity: Tamil
Content (Plot):
<<<
Road movies are popular in the western, but rarely are such filmed made india. It is a genre in which the main character or characters leave home to travel from place to place. They usually leave home to escape their current live and meet with many adventure that have a profound impact not only on the lives but also on those they came across during their travel.
Come to the movie story: A young woman Kalpana leaves home, because of her interest in a movie career which is kindled by a group of crook's. (Kallapart Natarajan) shoots a member of his gang which is she witness. To, escape them, she jumps onto a running bus going from madras to Pondicherry and then the fun starts. The gangsters engage a man who boards the bus the young women is into eliminate her.
However, a young man also gets in(Ravichandran). He too travels on the bus with her pals and realise that the young women is in trouble. He takes up the task of saving her and in the process falls in love with her. In the end it turns out that he is her prospective bridegroom to avoid whom she leaves home. They are many subplots involving interesting characters who travel on the same bus.
>>>

chunk_id: 31040_0
doc_id: 31040
Title: Aridhu Aridhu
Release Year: 2010
Genre: Not specified
Director: K. R. Mathivannan
Cast: Harish Kalyan, Uttara Raj, Princess Jasmine
Origin/Ethnicity: Tamil
Content (Plot):
<<<
The film has been shot in Chennai, Australia and New Zealand. The film mainly deals about the terrorism and violence happening around the world.
>>>

chunk_id: 31489_0
doc_id: 31489
Title: Chennai 2 Singapore
Release Year: 2017
Genre: Romantic Comedy
Director: Abbas Akbar
Cast: Gokul Anand, Rajesh Balachandiran, Anju Kurian
Origin/Ethnicity: Tamil
Content (Plot):
<<<
The film is a comedy about a struggling Indian film maker, Harish, who makes his way to Singapore to look for investors to finance his dream movie. However, a series of misfortunes foils his plans and he also ends up losing his passport. At a loss, Harish meets Vaanambaadi, an eccentric Singaporean cameraman. With Vaanambaadi's advice and help, Harish almost realises his dream. But then he meets a girl who is dying from cancer and gets swept up in a number of hilarious but serious situations. Without the cliched romance in Indian films, the story takes a light and humorous turn into a madcap journey about life, death and everything else.[4]
>>>

chunk_id: 28393_0
doc_id: 28393
Title: Kunthapura
Release Year: 2013
Genre: Drama, Period
Director: Joe Eshwar
Cast: Biyon, Priya Lal, Anu Hassan
Origin/Ethnicity: Malayalam
Content (Plot):
<<<
Structured on the point of view of three individuals, Krishnappa Narasimha Shashtri, an old Indian freedom fighter, Gowri Iyer, an Indian journalist and Emma Hall, a British banker. The film is set in the 1920s of colonial India and revolves around the lives of three groups of people, the peasants from a village called 'Kunthapura', the freedom fighters of the princely state of Mysore and a group of archeologists commissioned to do an excavation in the village of Kunthapura. The film shot in India and England is due for release in 2013.[5]
>>>

chunk_id: 27421_0
doc_id: 27421
Title: Cool...Sakkath Hot Maga
Release Year: 2011
Genre: Romance
Director: Ganesh
Cast: Ganesh, Aamna Sharif, Sadhu Kokila
Origin/Ethnicity: Kannada
Content (Plot):
<<<
The film has supposedly a comedy and romance mixed storyline. Ganesh plays a college going student in the film. The film has been shot in some picturesque locations such as Egypt, Dubai and Jordan and Middle East.[2]
>>>

chunk_id: 29402_0
doc_id: 29402
Title: Neelagiri Express
Release Year: 1968
Genre: Not specified
Director: Thirumalai-Mahalingam
Cast: Jaishankar, Vijaya Nirmala, Cho, Vijaya Lalitha
Origin/Ethnicity: Tamil
Content (Plot):
<<<
In Indian cinema it is usually stories from the epic or folk that are made in multiple language because of the wide appeal across regions. However, are story essentially a murder mystery about man killed on a moving train.
>>>

chunk_id: 26834_2
doc_id: 26834
Title: Mere Dost Picture Abhi Baki Hai
Release Year: 2012
Genre: Comedy
Director: Rajnish Raj Thakur
Cast: Suniel Shetty, Rajpal Yadav[49]
Origin/Ethnicity: Bollywood
Content (Plot):
<<<
What follows is a chain of events which gives the audience a never seen before insight into the modus operadi of how films are generally made in Bollywood. In a nutshell, Picture Abhi Baki Hai will take you on a wild journey inside the dreamy world of Bollywood where you will see the reality of the largest film industry of the world. It won't just make you laugh; it will make you appreciate the efforts of those who try to entertain you every minute.
>>>

</details>

---